# Analyzing Orangetheory's Customer Sentiment on Lift45 and HR Metrics

## Setup for Extracting Reddit posts and comments

### Importing relevant packages

In [198]:
import praw
import time
import json
import requests
import csv
import pandas as pd
from datetime import datetime, timedelta
import codecs
import os
from dotenv import load_dotenv

### Initializing a Reddit object and defining the subreddit

In [2]:
load_dotenv()

reddit = praw.Reddit(
   client_id=os.getenv("REDDIT_CLIENT_ID"),
   client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
   user_agent=os.getenv("REDDIT_USER_AGENT"),
   username=os.getenv("REDDIT_USERNAME"),
   password=os.getenv("REDDIT_PASSWORD")
)

subreddit = reddit.subreddit("orangetheory")

# Customer Sentiment about Lift45:
## Fetching all Reddit posts, comments, and replies about Lift45 from last week

In [43]:
keywords = ['lift45', 'lift 45']
one_week_ago = int(time.time()) - 7 * 24 * 60 * 60

In [44]:
def fetch_comments(comment, posts):
    # Add comment to posts
    decoded_comment = codecs.unicode_escape_decode(comment.body)[0]
    clean_comment = decoded_comment.replace('’', "'")
    clean_comment = clean_comment.encode('ascii', 'ignore').decode()
    
    posts.append({
        'date': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        'author': comment.author.name if comment.author else 'Deleted',
        'title': f'Re: {post.title}',
        'body': clean_comment.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    })

    # Process replies to the comment
    for reply in comment.replies:
        fetch_comments(reply, posts)

posts = []

for post in subreddit.new(limit = 1000):
    if any(keyword in post.title.lower() for keyword in keywords):
        if post.created_utc >= one_week_ago:
            decoded_text = codecs.unicode_escape_decode(post.selftext)[0]
            clean_text = decoded_text.replace('’', "'")
            clean_text = clean_text.encode('ascii', 'ignore').decode()
            
            posts.append({
                'date': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                'author': post.author.name if post.author else 'Deleted',
                'title': post.title,
                'body': clean_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
            })
            
            for comment in post.comments:
                fetch_comments(comment, posts)

/var/folders/nl/4h1npm454flgnps4vlz6y_700000gn/T/ipykernel_54082/3147963917.py:3: DeprecationWarning: invalid escape sequence '\~'
  decoded_comment = codecs.unicode_escape_decode(comment.body)[0]


In [45]:
len(posts)

81

In [48]:
posts[0]

{'date': '2023-07-22',
 'author': 'moonrivervoyages',
 'title': 'Regular class into Lift45 in place of a 90 minute?',
 'body': 'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think!'}

## ChatGPT API Setup
### Load the API key and relevant Python libaries.

In [7]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [8]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Determine categories of feedback based on last week's posts and comments

In [46]:
# Combine all post bodies into one string
all_posts = " ###END_OF_COMMENT### ".join([post['body'] for post in posts])

In [47]:
all_posts

'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think! ###END_OF_COMMENT### I prefer to do the Lift45 before the regular 60. But yes its doable. ###END_OF_COMMENT### Oooo yes smart idea however that isn\'t an option for me because of my work schedule! ###END_OF_COMMENT### Just know if you have a limited number of classes going the Lift + 60 will count as two classes and doing the 90 only counts as one. ###END_OF_COMMENT### Thank you so much! Good to know! ###END_OF_COMMENT### I found some information that could be relevant to your question or topic.      Take a look at [previous conversations about pairing Lift 45 with a regular class](https://www.google.com/search?q=regular+60+lift45+site:reddit.com/r/orangetheory)      _This is an 

In [56]:
prompt = f"""
Your role is to analyze and summarize the main topics of customer sentiment and feedback about the Lift45 \
course from a series of social media posts and comments. Lift45 is a course offered by the fitness company \
Orangetheory, sometimes referred to as OTF. 

You are presented with a compilation of these posts and comments, delimited by triple backticks. 

Analyze the text and identify the top topics that customers are positively and negatively commenting on. \
Focus only on the specific compliments, criticisms, and feedback about Lift45. Ignore any unrelated comments. \
* Use at most 4 words as the topic name and give a brief explanation of each. 
* Do not use snacks as one of the topics. 
* Give me 5 positive topics and 5 negative topics.

Social Media Posts: ```{all_posts}```
"""

response = get_completion(prompt)
print(response)

Positive topics:
1. Lift45 + Regular Class: Customers express that it is doable and enjoyable to pair Lift45 with a regular 60-minute class. They appreciate the opportunity to do both workouts and find it beneficial for their fitness goals.
2. Cardio Between Lifting: Customers mention that they enjoy the cardio exercises between lifting sets during Lift45. They find it effective and enjoyable for their workout.
3. Trainer's Opinion: Customers appreciate hearing from a NASM certified trainer who suggests doing the regular class before Lift45 for fat loss and vice versa for strength. They value the professional advice and find it helpful in planning their workouts.
4. Challenging Templates: Customers express their excitement and satisfaction with the challenging Lift45 templates. They appreciate the opportunity to lift heavy and feel the burn during the workout.
5. Slow and Heavy: Customers enjoy Lift45 templates that emphasize slow and heavy lifting. They find it effective and enjoyable

### Original list of 10 categories
1. **pairing with regulars**: Customers express that it is doable and enjoyable to pair Lift45 with a regular 60-minute class. They appreciate the opportunity to do both workouts and find it beneficial for their fitness goals.

2. **cardio**: Customers mention that they enjoy the cardio exercises between lifting sets during Lift45. They find it effective and enjoyable for their workout.

3. **coaching**: Customers appreciate the coaches moving around and correcting form during the class. Customers appreciate hearing from a NASM certified trainer who suggests doing the regular class before Lift45 for fat loss and vice versa for strength. They value the professional advice and find it helpful in planning their workouts.

4. **intensity**: Customers express their excitement and satisfaction with the challenging Lift45 templates. They appreciate the opportunity to lift heavy and feel the burn during the workout.

5. **slow lifting**: Customers enjoy Lift45 templates that emphasize slow and heavy lifting. They find it effective and enjoyable for their workout.

6. **longer blocks**: Customers like the longer blocks and minimal switching between exercises, as it allows them to focus and go at their own pace.

7. **limited availability**: Customers mention that the schedule for Lift45 classes does not always align with their availability. They express frustration with long waitlists and difficulty incorporating Lift45 into their schedule.

8. **exercise dislikes**: Customers express dislike for certain exercises such as the pop squats in the Lift45 template. They find it challenging and uncomfortable for their knees.

9. **repetitive templates**: Customers express a desire for more variety in the Lift45 templates. For example, they have done hip bridge exercises multiple times in a short period.

10. **lacking continuity**: Some customers mention that they prefer a 90-minute class over Lift45 because of the continuity it offers. They feel that the breaks between exercises in Lift45 disrupt the flow of their workout.

### Narrowing down categories that are too similar:

**exercise content**
* repetitive exercise and dislikes: Customers express a desire for more variety in the Lift45 templates. For example, they have done hip bridge exercises multiple times in a short period. Customers also express dislike for certain exercises. For example, they find the pop squats challenging and uncomfortable for their knees.
* cardio: Customers mention that they enjoy the cardio exercises between lifting sets during Lift45. They find it effective and enjoyable for their workout.
* intensity: Customers express their excitement and satisfaction with the challenging Lift45 templates. They appreciate the opportunity to lift heavy and feel the burn during the workout.
* longer blocks: Customers like the longer blocks and minimal switching between exercises, as it allows them to focus and go at their own pace. Customers enjoy Lift45 templates that emphasize slow and heavy lifting. They find it effective and enjoyable for their workout.
* lacking continuity: Some customers mention that they prefer a 90-minute class over Lift45 because of the continuity it offers. They feel that the breaks between exercises in Lift45 disrupt the flow of their workout.

**course relationship**
* How should a customer choose what classes to take and in what order. Some courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

**course availability**
* Customers mention that the schedule for Lift45 classes does not always align with their availability. They express frustration with long waitlists and difficulty incorporating Lift45 into their schedule.
    
**coaching**
* Customers appreciate the coaches moving around and correcting form during the class. Customers appreciate hearing from a NASM certified trainer who suggests doing the regular class before Lift45 for fat loss and vice versa for strength. They value the professional advice and find it helpful in planning their workouts.

**perceived price value**
* how course credits are used and counted

## Writing a prompt for creating a list of posts and comments that are about customer sentiment of Lift45

### Using the first 10 posts and comments for testing the prompt

In [71]:
test_posts = posts[:10]

In [107]:
test_posts

[{'date': '2023-07-22',
  'author': 'moonrivervoyages',
  'title': 'Regular class into Lift45 in place of a 90 minute?',
  'body': 'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think!'},
 {'date': '2023-07-22',
  'author': 'runningJD_82',
  'title': 'Re: Regular class into Lift45 in place of a 90 minute?',
  'body': 'I prefer to do the Lift45 before the regular 60. But yes its doable.'},
 {'date': '2023-07-22',
  'author': 'moonrivervoyages',
  'title': 'Re: Regular class into Lift45 in place of a 90 minute?',
  'body': "Oooo yes smart idea however that isn't an option for me because of my work schedule!"},
 {'date': '2023-07-22',
  'author': 'TuesGirl',
  'title': 'Re: Regular class into Lift45 in place of a 90 minute?',
  'body':

<!-- Example post for posts that do not contain customer sentiment, compliments, criticisms, and feedback:
    'date': '2023-07-22',
    'author': 'moonrivervoyages',
    'title': 'Regular class into Lift45 in place of a 90 minute?',
    'body': 'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. \
     My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? \
     I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what \
     others think!'
     
    Example response for above post: 'Irrelevant feedback.' -->

### Prompt Template for Lift45 on Reddit

In [105]:
json_objects = []

for i in range(len(test_posts)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    the course Lift45, which is offered by the fitness company Orangetheory. Orangetheory is sometimes referred \
    to as OTF. 

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{posts[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{test_posts[i]['date']}#
    
    2. topic: 'Lift45'

    3. title: ##{test_posts[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{test_posts[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about Lift45.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about Lift45. Do not extract any other texts.   

    9. category: 
    
        5 categories of feedback are as follows:

        - exercise content: How effective are the exercrises in Lift45 in terms of activity type, \
        intensity, and pace. 
        
        - course relationship: How should a customer choose what classes to take and in what order. \ Some \
        courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

        - course availability: How does scheduling and capacity constraints affect customers' ability to \
        join Lift45 classes.

        - coaching: How professional and helpful the trainer is.

        - perceived price value: how course credits are used and counted.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'exercise content', course relationship', 'course availability', 'coaching', \
    and 'perceived price value'. Use "other" if the post/comment does not fall into any of the 5 categories. \
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i, response, "\n")
    
    # Load the response string as a dictionary and add to the list
    json_objects.append(json.loads(response))

    time.sleep(20)

0 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "moonrivervoyages",
    "sentiment": "neutral",
    "summary": "The author is seeking opinions on whether it is doable to replace a 90-minute class with a Lift45 class and wants to know what others think.",
    "text": "Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think!",
    "category": "course relationship"
} 

1 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "runningJD_82",
    "sentiment": "positive",
    "summary": "The author prefers to do Lift45 before the regular 60-minute class and finds it doable.",
    "text": "I prefer to do the Lift45 before the regular 60. But yes its doable.",
    "category": "course relat

## Fetching all Reddit posts, comments, and replies about Lift45 from the last 3 months

In [194]:
keywords = ['lift45', 'lift 45']
three_months_ago = int(time.time()) - 90 * 24 * 60 * 60

In [195]:
posts = []

for post in subreddit.new(limit = 2000):
    if any(keyword in post.title.lower() for keyword in keywords):
        if post.created_utc >= three_months_ago:
            decoded_text = codecs.unicode_escape_decode(post.selftext)[0]
            clean_text = decoded_text.replace('’', "'")
            clean_text = clean_text.encode('ascii', 'ignore').decode()
            
            posts.append({
                'date': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                'author': post.author.name if post.author else 'Deleted',
                'title': post.title,
                'body': clean_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
            })
            
            for comment in post.comments:
                fetch_comments(comment, posts)

/var/folders/nl/4h1npm454flgnps4vlz6y_700000gn/T/ipykernel_54082/3147963917.py:3: DeprecationWarning: invalid escape sequence '\~'
  decoded_comment = codecs.unicode_escape_decode(comment.body)[0]
/var/folders/nl/4h1npm454flgnps4vlz6y_700000gn/T/ipykernel_54082/3147963917.py:3: DeprecationWarning: invalid escape sequence '\_'
  decoded_comment = codecs.unicode_escape_decode(comment.body)[0]
/var/folders/nl/4h1npm454flgnps4vlz6y_700000gn/T/ipykernel_54082/3147963917.py:3: DeprecationWarning: invalid escape sequence '\*'
  decoded_comment = codecs.unicode_escape_decode(comment.body)[0]


In [196]:
len(posts)

593

In [193]:
posts[-1]

{'date': '2023-06-08',
 'author': 'caitplusate',
 'title': 'Re: Lift45 TB1 - week of 6/5',
 'body': 'I am so sick of lateral lunges!'}

In [125]:
#posts, comments, and replies about Lift45 from the last 3 months
ThreeMonthPosts_Lift45 = posts

### Running the prompt with ThreeMonthPosts_Lift45

In [127]:
json_objects = []

for i in range(len(ThreeMonthPosts_Lift45)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    the course Lift45, which is offered by the fitness company Orangetheory. Orangetheory is sometimes referred \
    to as OTF. 

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{ThreeMonthPosts_Lift45[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{ThreeMonthPosts_Lift45[i]['date']}#
    
    2. topic: 'Lift45'

    3. title: ##{ThreeMonthPosts_Lift45[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{ThreeMonthPosts_Lift45[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about Lift45.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about Lift45. Do not extract any other texts.   

    9. category: 
    
        5 categories of feedback are as follows:

        - exercise content: How effective are the exercrises in Lift45 in terms of activity type, \
        intensity, and pace. 
        
        - course relationship: How should a customer choose what classes to take and in what order. \ Some \
        courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

        - course availability: How does scheduling and capacity constraints affect customers' ability to \
        join Lift45 classes.

        - coaching: How professional and helpful the trainer is.

        - perceived price value: how course credits are used and counted.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'exercise content', course relationship', 'course availability', 'coaching', \
    and 'perceived price value'. Use "other" if the post/comment does not fall into any of the 5 categories. \
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i, response, "\n")
    
    # Load the response string as a dictionary and add to the list
    json_objects.append(json.loads(response))

0 {
  "date": "2023-07-24",
  "topic": "Lift45",
  "title": "Lift 45 TB1 Week of 7/24/23",
  "source": "Reddit",
  "author": "Crafty-Persimmon1857",
  "sentiment": "positive",
  "summary": "The author is seeking feedback on the exercises in Lift45 and expresses gratitude for any edits or corrections.",
  "text": "Help a sister out; some of these exercises I feel I put in the wrong block or mislabeled. Let me know what needs an edit. Thanks! Happy lifting. :)",
  "category": "exercise content"
} 

1 {
    "date": "2023-07-24",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 Week of 7/24/23",
    "source": "Reddit",
    "author": "Fickle-Huckleberry-8",
    "sentiment": "positive",
    "summary": "The post expresses positive customer sentiment and compliments about Lift45.",
    "text": "I was just wishing for this sort of intel, thank you!!!! Looks awesome!!!!",
    "category": "other"
} 

2 {
  "date": "2023-07-24",
  "topic": "Lift45",
  "title": "Re: Lift 45 TB1 Week of 7/24/23"

12 {
  "date": "2023-07-23",
  "topic": "Lift45",
  "title": "Re: Two different people borrowed from my station today. Lift45 format small rant.",
  "source": "Reddit",
  "author": "char227",
  "sentiment": "positive",
  "summary": "The author expresses a positive sentiment about Lift45, mentioning that everyone at their studio is nice and asks before borrowing.",
  "text": "We do that at our studio but everyone is really nice about it and asks before.",
  "category": "other"
} 

13 {
    "date": "2023-07-23",
    "topic": "Lift45",
    "title": "Re: Two different people borrowed from my station today. Lift45 format small rant.",
    "source": "Reddit",
    "author": "Twirly16us",
    "sentiment": "positive",
    "summary": "The author shares a positive experience with Lift45, mentioning the availability of 12 floor and 12 rowing stations, helpful coaches, and encouragement for weight swapping.",
    "text": "My studio literally does 12 floor 12 rowing stations and I have been to full 

27 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "moonrivervoyages",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment about Lift45.",
    "text": "Oh awesome! Thank you! What are your fav between class snacks?",
    "category": "other"
} 

28 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "jenniferlynn5454",
    "sentiment": "positive",
    "summary": "The author expresses their love for Aussie bites, protein bars, and cashew clusters from Costco.",
    "text": "I love Aussie bites, protein bars or cashew clusters, all from Costco",
    "category": "other"
} 

29 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "

45 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "moonrivervoyages",
    "sentiment": "positive",
    "summary": "The post expresses gratitude and positive sentiment towards Lift45.",
    "text": "Thank you!",
    "category": "other"
} 

46 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "happyhiker1118",
    "sentiment": "positive",
    "summary": "The author expresses satisfaction with the Lift45 class and mentions taking it instead of a 90-minute class.",
    "text": "I usually do a 2G followed by lift once a week. I always take the day off from Orangetheory the next day, though - so if the next day is a benchmark or specialty class I really want to do, Ill just do the lift.",
    "category": "course relationship"
} 

47 {
    "date": "2023-07-22",
    "topic":

65 {
  "date": "2023-07-20",
  "topic": "Lift45",
  "title": "Re: 7/17/23 Lift 45 Total Body 3",
  "source": "Reddit",
  "author": "Dutchsally",
  "sentiment": "positive",
  "summary": "The author provides positive feedback about Lift45, suggesting to lift lighter, go at your own pace, take rests when needed, pay attention to form, and modify when necessary.",
  "text": "Just lift a little lighter, go at your own pace, take rests when you need them, pay attention to form rather than rushing through, and modify when needed. (I should  pay more attention to my own advice, haha). I did lift after long breaks, and didnt die. I found it better for reintroduction than regular classes.",
  "category": "exercise content"
} 

66 {
    "date": "2023-07-20",
    "topic": "Lift45",
    "title": "Re: 7/17/23 Lift 45 Total Body 3",
    "source": "Reddit",
    "author": "GiftEmergency4288",
    "sentiment": "positive",
    "summary": "The author had a positive experience with Lift45, finding it chaot

...

102 {
    "date": "2023-07-17",
    "topic": "Lift45",
    "title": "Re: 07/17/2023 Lift 45 Total Body 1",
    "source": "Reddit",
    "author": "kenzika",
    "sentiment": "positive",
    "summary": "The post expresses excitement and gratitude towards Lift45.",
    "text": "Yay walking out the door to this right now thank you so much!",
    "category": "other"
} 

103 {
    "date": "2023-07-17",
    "topic": "Lift45",
    "title": "Re: 07/17/2023 Lift 45 Total Body 1",
    "source": "Reddit",
    "author": "dudeitsursula",
    "sentiment": "positive",
    "summary": "Excited for tomorrow's Lift45 class.",
    "text": "This looks great! So excited for tomorrow",
    "category": "other"
} 

104 {
    "date": "2023-07-17",
    "topic": "Lift45",
    "title": "Re: 07/17/2023 Lift 45 Total Body 1",
    "source": "Reddit",
    "author": "debgrasso",
    "sentiment": "positive",
    "summary": "Looks like a really good one. Thanks!",
    "text": "Looks like a really good one. Thanks!",
    "

119 {
  "date": "2023-07-16",
  "topic": "Lift45",
  "title": "Re: When to incorporate more Lift45?",
  "source": "Reddit",
  "author": "thisisjustmylife7",
  "sentiment": "positive",
  "summary": "The author shares their personal journey of weight gain and mental health issues, expressing gratitude for feeling healthier and not gaining more weight.",
  "text": "No big movement yet but I try to tell myself that at least Im not gaining and Im feeling healthier???",
  "category": "other"
} 

120 {
  "date": "2023-07-16",
  "topic": "Lift45",
  "title": "Re: When to incorporate more Lift45?",
  "source": "Reddit",
  "author": "figuringitout25",
  "sentiment": "negative",
  "summary": "The author expresses frustration about not losing weight quickly and missing the enjoyment of watching the scale move slowly when gaining weight.",
  "text": "I keep telling myself I didnt put it on overnight so Im not going to lose it overnight but damn did I have a lot more fun watching the scale move slow

133 {
  "date": "2023-07-11",
  "topic": "Lift45",
  "title": "Lift 45s",
  "source": "Reddit",
  "author": "Melodic-Entrance577",
  "sentiment": "positive",
  "summary": "The author expresses enjoyment in lifting/resistance training over treadmills and considers signing up for back-to-back Lift45 classes.",
  "text": "I truly enjoy lifting/resistance training over the treadmills. There is a day this week where there is a normal class at 8:45am then a lift at 10am. Im thinking of signing up for both.",
  "category": "exercise content"
} 

134 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45s",
    "source": "Reddit",
    "author": "MikkyG01",
    "sentiment": "positive",
    "summary": "The author enjoys pairing Lift45 with a 60-minute class, prefers doing Lift45 first for muscle building, and finds it easier to run tired than lift tired. They also appreciate starting on the floor to go home after rowing.",
    "text": "I like to pair a lift 45 with a 60 som

149 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45s",
    "source": "Reddit",
    "author": "OTFLyfer",
    "sentiment": "positive",
    "summary": "The post/comment expresses a positive sentiment about Lift45.",
    "text": "Yes it is, enjoy!!",
    "category": "other"
} 

150 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45s",
    "source": "Reddit",
    "author": "jplikescoffee",
    "sentiment": "positive",
    "summary": "The author loves doing lifts and a 60 on Monday mornings, considering it the best way to start the week.",
    "text": "I love doing lifts and then a 60 Monday mornings, best way to start the week!",
    "category": "exercise content"
} 

151 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45s",
    "source": "Reddit",
    "author": "renesme3102",
    "sentiment": "positive",
    "summary": "The author expresses a positive sentiment towards Lift45, mentioning that they do it thems

ServiceUnavailableError: The server is overloaded or not ready yet.

#### Since the server was overloaded, the list ThreeMonthPosts_Lift45 was cut to run the remaining posts.

In [129]:
json_objects[-1]

{'date': '2023-07-10',
 'topic': 'Lift45',
 'title': 'Lift 45 Total Body 3 (TB3 - Week of 7/10/2023',
 'source': 'Reddit',
 'author': 'StevenOTF',
 'sentiment': 'positive',
 'summary': 'The post provides details about the Lift45 class, including the exercises and structure, and the author seems satisfied with the format.',
 'text': "This should be close, but not perfect. I'll edit if anyone remembers the missing details. There are two 14 minute main blocks plus a shorter core block. Each of the main blocks has a repeating exercise that you will do every 2.5 minutes. The coach will call for this and you pause where you're at in the other exercises, do the 10 reps, then pick up where you left off. Block 1 (14 minutes) 10x - TRX High Row Explosive Tempo (Repeat every 2:30) 8x Each Side - Goblet Lateral Lunge 8x Each Arm - Hollow Hold Neutral Grip Chest Press 8x Each Leg - Single Leg Bridge Start with the TRX high rows, repeat all other exercises until time is called, but pause where you'r

In [130]:
Cut_ThreeMonthPosts_Lift45 = ThreeMonthPosts_Lift45[158:]

In [132]:
for i in range(len(Cut_ThreeMonthPosts_Lift45)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    the course Lift45, which is offered by the fitness company Orangetheory. Orangetheory is sometimes referred \
    to as OTF. 

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{Cut_ThreeMonthPosts_Lift45[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{Cut_ThreeMonthPosts_Lift45[i]['date']}#
    
    2. topic: 'Lift45'

    3. title: ##{Cut_ThreeMonthPosts_Lift45[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{Cut_ThreeMonthPosts_Lift45[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about Lift45.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about Lift45. Do not extract any other texts.   

    9. category: 
    
        5 categories of feedback are as follows:

        - exercise content: How effective are the exercrises in Lift45 in terms of activity type, \
        intensity, and pace. 
        
        - course relationship: How should a customer choose what classes to take and in what order. \ Some \
        courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

        - course availability: How does scheduling and capacity constraints affect customers' ability to \
        join Lift45 classes.

        - coaching: How professional and helpful the trainer is.

        - perceived price value: how course credits are used and counted.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'exercise content', course relationship', 'course availability', 'coaching', \
    and 'perceived price value'. Use "other" if the post/comment does not fall into any of the 5 categories. \
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i+158, response, "\n")
    
    # Load the response string as a dictionary and add to the list
    json_objects.append(json.loads(response))

158 {
    "date": "2023-07-13",
    "topic": "Lift45",
    "title": "Re: Lift 45 Total Body 3 (TB3 - Week of 7/10/2023)",
    "source": "Reddit",
    "author": "rocroc00",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive customer sentiment about Lift45.",
    "text": "You had me at SA alternating snatch",
    "category": "exercise content"
} 

159 {
    "date": "2023-07-10",
    "topic": "Lift45",
    "title": "Re: Lift 45 Total Body 3 (TB3 - Week of 7/10/2023)",
    "source": "Reddit",
    "author": "strayainind",
    "sentiment": "positive",
    "summary": "This was an amazing class. High burn for a lift.",
    "text": "This was an amazing class. High burn for a lift.",
    "category": "exercise content"
} 

160 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45 Total Body 3 (TB3 - Week of 7/10/2023",
    "source": "Reddit",
    "author": "TriciaMc99",
    "sentiment": "positive",
    "summary": "Fantastic endurance lift class

177 {
    "date": "2023-07-10",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 - week of 7/10",
    "source": "Reddit",
    "author": "EMAW262",
    "sentiment": "neutral",
    "summary": "The post has a neutral sentiment.",
    "text": "I am with ya.",
    "category": "other"
} 

178 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 - week of 7/10",
    "source": "Reddit",
    "author": "theekp",
    "sentiment": "positive",
    "summary": "The author expresses agreement and satisfaction with the Lift45 class, mentioning their preference for lifting heavy weights and getting cardio in regular classes or outside the studio.",
    "text": "agree! this is a class where I'll go a little rogue and swap out some of the power/hop moves with something that uses similar muscles and a weight. I'm there to lift as heavy as I can. I'll get my cardio in regular class or outside of the studio.",
    "category": "exercise content"
} 

179 {
    "date": "2023-07-1

198 {
    "date": "2023-07-12",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 - week of 7/10",
    "source": "Reddit",
    "author": "Beautiful_Cucumber18",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment about Lift45 and provides information on finding the most current workouts in the intel.",
    "text": "If you mean intel, keep scrolling down looking for the correct dates & TB2/3 labels. TB2 is definitely in there for this week. I just did it today and viewed the intel last night. The most current workouts aren't always at the top, I was told they have to get more upvotes to get boosted up there.",
    "category": "course availability"
} 

199 {
    "date": "2023-07-11",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 - week of 7/10",
    "source": "Reddit",
    "author": "AdLost1161",
    "sentiment": "neutral",
    "summary": "The post is asking if Lift45 is similar to Orange 45, indicating a neutral sentiment.",
    "text": "

215 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "AmanLock",
    "sentiment": "neutral",
    "summary": "The post is neutral and discusses the possibility of the Lift45 format varying depending on the studio.",
    "text": "I'd guess that may depend on the studio.",
    "category": "other"
} 

216 {
    "date": "2023-07-16",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "soneg",
    "sentiment": "negative",
    "summary": "The post expresses frustration about the timing of Lift45 classes conflicting with work schedules.",
    "text": "Really hard for those of us that have work.",
    "category": "course availability"
} 

217 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "afterburnkid",
    "sentiment": "negative",
    "summary": "The author expresses

233 {
    "date": "2023-07-07",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "Sucker4theRower",
    "sentiment": "positive",
    "summary": "The post/comment expresses a positive sentiment about Lift45.",
    "text": "Well stated my friend! Truth!",
    "category": "other"
} 

234 {
  "date": "2023-07-06",
  "topic": "Lift45",
  "title": "Re: Changes to Lift45 format?",
  "source": "Reddit",
  "author": "Burning-the-wagon",
  "sentiment": "positive",
  "summary": "The author expresses enthusiasm and satisfaction with the lower body burn experienced during Lift45.",
  "text": "I did TB2 today (after my 2G) and my thighs are still feeling it- I love a good lower body burn.",
  "category": "exercise content"
} 

235 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "jenniferlynn5454",
    "sentiment": "positive",
    "summary": "The author expr

252 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "schulzch13",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive customer sentiment about Lift45.",
    "text": "The post/comment does not provide any specific feedback or sentiment about Lift45.",
    "category": "other"
} 

253 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "Distinct-Hold-5836",
    "sentiment": "positive",
    "summary": "The author expresses a positive sentiment towards the changes made to Lift45.",
    "text": "OT has needed to do this for awhile. Change is good.",
    "category": "other"
} 

254 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "Kristen1604",
    "sentiment": "positive",
    "summary": "The author expresses exc

270 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "kwilson7499",
    "sentiment": "neutral",
    "summary": "The post is neutral and the author is seeking information about leg day or full body workouts in Lift45.",
    "text": "Would love to know when leg day or full body",
    "category": "exercise content"
} 

271 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "Skittlebrau77",
    "sentiment": "positive",
    "summary": "The author expresses their love for Lift45.",
    "text": "Yesss I love Lift",
    "category": "exercise content"
} 

272 {
    "date": "2023-07-06",
    "topic": "Lift45",
    "title": "Re: Changes to Lift45 format?",
    "source": "Reddit",
    "author": "Icarusgurl",
    "sentiment": "neutral",
    "summary": "The author expresses a neutral sentiment about the availability of Lift45 classe

287 {
  "date": "2023-07-03",
  "topic": "Lift45",
  "title": "Re: Lift 45 TB 1 week of July 3rd",
  "source": "Reddit",
  "author": "MaximumUsual880",
  "sentiment": "neutral",
  "summary": "The author recalls a workout session, unsure if it was during a row exercise or Lift45, and mentions the limited movements in Lift45.",
  "text": "It also could have been during a row exercise, but I feel like I remember it being Lift because there are not many movements that are not split between 'floor' and 'row' side.",
  "category": "exercise content"
} 

288 {
    "date": "2023-07-03",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB 1 week of July 3rd",
    "source": "Reddit",
    "author": "Burning-the-wagon",
    "sentiment": "neutral",
    "summary": "The post has a neutral sentiment and mentions power sit-ups with the toss.",
    "text": "And the power sit-ups with the toss",
    "category": "exercise content"
} 

289 {
    "date": "2023-07-03",
    "topic": "Lift45",
    "title": "Re

ServiceUnavailableError: The server is overloaded or not ready yet.

#### Again, since the server was overloaded, the list ThreeMonthPosts_Lift45 was cut to run the remaining posts.

In [133]:
json_objects[-1]

{'date': '2023-06-26',
 'topic': 'Lift45',
 'title': 'Re: Lift 45 TB1 Week of 6/26',
 'source': 'Reddit',
 'author': 'Burning-the-wagon',
 'sentiment': 'negative',
 'summary': "The author is rethinking their plan to attend a Lift45 class and expresses that it won't bring them joy.",
 'text': 'I love lift45 classes but this doesnt look like it will bring me joy!',
 'category': 'exercise content'}

In [135]:
SecondCut_ThreeMonthPosts_Lift45 = ThreeMonthPosts_Lift45[307:]

for i in range(len(SecondCut_ThreeMonthPosts_Lift45)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    the course Lift45, which is offered by the fitness company Orangetheory. Orangetheory is sometimes referred \
    to as OTF. 

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{SecondCut_ThreeMonthPosts_Lift45[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{SecondCut_ThreeMonthPosts_Lift45[i]['date']}#
    
    2. topic: 'Lift45'

    3. title: ##{SecondCut_ThreeMonthPosts_Lift45[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{SecondCut_ThreeMonthPosts_Lift45[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about Lift45.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about Lift45. Do not extract any other texts.   

    9. category: 
    
        5 categories of feedback are as follows:

        - exercise content: How effective are the exercrises in Lift45 in terms of activity type, \
        intensity, and pace. 
        
        - course relationship: How should a customer choose what classes to take and in what order. \ Some \
        courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

        - course availability: How does scheduling and capacity constraints affect customers' ability to \
        join Lift45 classes.

        - coaching: How professional and helpful the trainer is.

        - perceived price value: how course credits are used and counted.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'exercise content', course relationship', 'course availability', 'coaching', \
    and 'perceived price value'. Use "other" if the post/comment does not fall into any of the 5 categories. \
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i+307, response, "\n")
    
    # Load the response string as a dictionary and add to the list
    json_objects.append(json.loads(response))

307 {
  "date": "2023-06-26",
  "topic": "Lift45",
  "title": "Re: Lift 45 TB1 Week of 6/26",
  "source": "Reddit",
  "author": "Lgotts",
  "sentiment": "positive",
  "summary": "The post expresses positive sentiment and provides feedback on the exercises in Lift45.",
  "text": "Almost perfect! The first one was a push up with alt low row. The last in that block was a neutral grip push press. Second starting exercise was an alt chest press with leg extension. Thanks for posting!",
  "category": "exercise content"
} 

308 {
    "date": "2023-06-26",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 Week of 6/26",
    "source": "Reddit",
    "author": "Crafty-Persimmon1857",
    "sentiment": "positive",
    "summary": "The post expresses gratitude and indicates that an issue has been fixed.",
    "text": "Thank you! Fixed :)",
    "category": "other"
} 

309 {
    "date": "2023-06-29",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1 Week of 6/26",
    "source": "Reddit",
    "aut

325 {
    "date": "2023-06-22",
    "topic": "Lift45",
    "title": "Re: Lift45",
    "source": "Reddit",
    "author": "blahlalala1234",
    "sentiment": "negative",
    "summary": "The author expresses frustration about the difficulty in seeing the exercises on the screen during Lift45 classes.",
    "text": "Or how about the days they put 12 exercises on the screen in a row and you can barely see them",
    "category": "exercise content"
} 

326 {
    "date": "2023-06-23",
    "topic": "Lift45",
    "title": "Re: Lift45",
    "source": "Reddit",
    "author": "blahlalala1234",
    "sentiment": "neutral",
    "summary": "The post/comment does not express a clear positive or negative sentiment about Lift45.",
    "text": "More jumping today squat jumps",
    "category": "exercise content"
} 

327 {
    "date": "2023-06-22",
    "topic": "Lift45",
    "title": "Re: Lift45",
    "source": "Reddit",
    "author": "jenniferlynn5454",
    "sentiment": "negative",
    "summary": "The author

342 {
  "date": "2023-06-25",
  "topic": "Lift45",
  "title": "Re: Lift45",
  "source": "Reddit",
  "author": "cmh8615",
  "sentiment": "positive",
  "summary": "The author expresses the benefits of doing full body lifts in Lift45 for longevity and balance, initially being doubtful but now convinced.",
  "text": "Crazy amount of benefits to do full body lifts vs. muscle group splits. One of the main reasons being that most people dont have the ability to do enough strength-focused workouts per week to stay balanced in their body. Its definitely better for longevity and balance to have total body, I was doubtful at first though!",
  "category": "exercise content"
} 

343 {
  "date": "2023-06-19",
  "topic": "Lift45",
  "title": "Lift 45 TB1, week of June 19",
  "source": "Reddit",
  "author": "Tasty-Tax2840",
  "sentiment": "positive",
  "summary": "The author enjoyed the Lift45 workout, finding it to be a great workout with lots of opportunities for heavy weights.",
  "text": "This is 

359 {
  "date": "2023-06-19",
  "topic": "Lift45",
  "title": "Re: Lift 45 TB1, week of June 19",
  "source": "Reddit",
  "author": "ZeeKayYou",
  "sentiment": "negative",
  "summary": "This post/comment expresses a negative sentiment about Lift45, stating that it is irregular compared to other lift classes.",
  "text": "Most lift classes are not like this. Dont let it deter you from trying again, some templates are really awesome heavy lifting. This one is very irregular.",
  "category": "exercise content"
} 

360 {
    "date": "2023-06-20",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1, week of June 19",
    "source": "Reddit",
    "author": "jenniferlynn5454",
    "sentiment": "negative",
    "summary": "The author expresses dissatisfaction with the Lift45 template.",
    "text": "Idk what they were thinking with this one",
    "category": "exercise content"
} 

361 {
  "date": "2023-06-20",
  "topic": "Lift45",
  "title": "Re: Lift 45 TB1, week of June 19",
  "source": "Redd

376 {
    "date": "2023-06-24",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1, week of June 19",
    "source": "Reddit",
    "author": "queef-beast420",
    "sentiment": "neutral",
    "summary": "The post is asking about the workout schedule for Lift45 and whether Sundays count as part of the week.",
    "text": "Will this likely be the TB1 for tomorrow? Do Sundays count as part of the week for this workout?",
    "category": "other"
} 

377 {
    "date": "2023-06-25",
    "topic": "Lift45",
    "title": "Re: Lift 45 TB1, week of June 19",
    "source": "Reddit",
    "author": "cleenk",
    "sentiment": "negative",
    "summary": "The post/comment expresses a negative sentiment about Lift45.",
    "text": "Soooo...NOT a lift class..",
    "category": "exercise content"
} 

378 {
    "date": "2023-06-17",
    "topic": "Lift45",
    "title": "Lift45",
    "source": "Reddit",
    "author": "Lala_H_6",
    "sentiment": "positive",
    "summary": "The author is seeking opinions on t

...

447 {
  "date": "2023-06-08",
  "topic": "Lift45",
  "title": "Re: Only doing Lift 45",
  "source": "Reddit",
  "author": "liftsinlulu",
  "sentiment": "positive",
  "summary": "The author loves Lift45 and finds it beneficial for strength training, building muscle, and feeling less drained from cardio.",
  "text": "I LOVE it. My goals are now more in strength training, building muscle, especially as a women 45y/o +. But I need the accountability of signing up for class. A side effect was that I dont feel as drained from all the cardio so Ill find myself more active after the workout. Win win for me.",
  "category": "exercise content"
} 

448 {
  "date": "2023-06-09",
  "topic": "Lift45",
  "title": "Re: Only doing Lift 45",
  "source": "Reddit",
  "author": "cbiancasea",
  "sentiment": "positive",
  "summary": "The author loves feeling strong and being able to focus on proper form in Lift45 classes, and expresses missing them when doing other classes.",
  "text": "I love feeling strong

462 {
    "date": "2023-06-08",
    "topic": "Lift45",
    "title": "Re: Only doing Lift 45",
    "source": "Reddit",
    "author": "Burning-the-wagon",
    "sentiment": "positive",
    "summary": "The author likes wearing the OTF tracker to keep track of their classes and stats, but they have learned to ignore it. They mention that their Apple Watch shows a higher burn during workouts. Despite having to use lighter weights due to a shoulder injury, they are still getting a good burn and have learned to slow down for more time under tension.",
    "text": "I like wearing it cause it keeps track of my classes in my stats that way- but Ive learned to ignore it. Also when I do a workout on my Apple Watch (functional ST or just ST) the burn is always higher. Yesterday after class someone asked me how I can do a double, and I was like cause my body doesnt feel like it just worked out. I went from check pressing 30s to being scared to go higher that 15-20 because of a shoulder injury. I am s

471 {
    "date": "2023-06-09",
    "topic": "Lift45",
    "title": "Re: Only doing Lift 45",
    "source": "Reddit",
    "author": "Sea_Signal_5739",
    "sentiment": "positive",
    "summary": "The author loves the Lift classes and focuses on them, also participating in 2 or 3Gs a couple of times a week.",
    "text": "I love the Lift classes and focus on them and do a couple of the 2 or 3Gs a couple of times a week.",
    "category": "exercise content"
} 

472 {
    "date": "2023-06-08",
    "topic": "Lift45",
    "title": "Re: Only doing Lift 45",
    "source": "Reddit",
    "author": "impulsive-puppy",
    "sentiment": "positive",
    "summary": "The author had a positive experience with Lift45, achieving more splat points compared to the regular classes.",
    "text": "I did my first lift 45 and achieved 7 splat points where I normally only able to get 3 during the full hour so far. Definitely going back to this class",
    "category": "exercise content"
} 

473 {
    "date": "20

487 {
  "date": "2023-06-08",
  "topic": "Lift45",
  "title": "Re: Only doing Lift 45",
  "source": "Reddit",
  "author": "Waste-Razzmatazz4147",
  "sentiment": "positive",
  "summary": "The author joined Lift45 for cardio and prefers the 2G and 3G classes, mentioning that they can use dumbbells and body weight at home.",
  "text": "I joined for cardio and prefer the 2G and 3G. I can just use dumb bells and body weight at home.",
  "category": "exercise content"
} 

488 {
    "date": "2023-06-08",
    "topic": "Lift45",
    "title": "Re: Only doing Lift 45",
    "source": "Reddit",
    "author": "Connect_Lynx_5723",
    "sentiment": "positive",
    "summary": "The author loves the Lift classes and enjoys doing them now that the weather is nicer.",
    "text": "Love the Lift classes",
    "category": "exercise content"
} 

489 {
    "date": "2023-06-08",
    "topic": "Lift45",
    "title": "Re: Only doing Lift 45",
    "source": "Reddit",
    "author": "Kindly-Might-1879",
    "sentimen

ServiceUnavailableError: The server is overloaded or not ready yet.

#### Again, since the server was overloaded, the list ThreeMonthPosts_Lift45 was cut to run the remaining posts.

In [136]:
json_objects[-1]

{'date': '2023-06-08',
 'topic': 'Lift45',
 'title': 'Re: Only doing Lift 45',
 'source': 'Reddit',
 'author': 'umwellshe',
 'sentiment': 'neutral',
 'summary': 'The author is curious about Lift45 classes but currently has a class pack membership.',
 'text': "Just curious what type of membership you currently have? I've been really curious about the Lift45 classes but I am only doing a class pack right now.",
 'category': 'other'}

In [137]:
ThirdCut_ThreeMonthPosts_Lift45 = ThreeMonthPosts_Lift45[495:]

for i in range(len(ThirdCut_ThreeMonthPosts_Lift45)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    the course Lift45, which is offered by the fitness company Orangetheory. Orangetheory is sometimes referred \
    to as OTF. 

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{ThirdCut_ThreeMonthPosts_Lift45[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{ThirdCut_ThreeMonthPosts_Lift45[i]['date']}#
    
    2. topic: 'Lift45'

    3. title: ##{ThirdCut_ThreeMonthPosts_Lift45[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{ThirdCut_ThreeMonthPosts_Lift45[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about Lift45.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about Lift45. Do not extract any other texts.   

    9. category: 
    
        5 categories of feedback are as follows:

        - exercise content: How effective are the exercrises in Lift45 in terms of activity type, \
        intensity, and pace. 
        
        - course relationship: How should a customer choose what classes to take and in what order. \ Some \
        courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

        - course availability: How does scheduling and capacity constraints affect customers' ability to \
        join Lift45 classes.

        - coaching: How professional and helpful the trainer is.

        - perceived price value: how course credits are used and counted.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'exercise content', course relationship', 'course availability', 'coaching', \
    and 'perceived price value'. Use "other" if the post/comment does not fall into any of the 5 categories. \
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i+495, response, "\n")
    
    # Load the response string as a dictionary and add to the list
    json_objects.append(json.loads(response))

495 {
  "date": "2023-06-09",
  "topic": "Lift45",
  "title": "Re: Only doing Lift 45",
  "source": "Reddit",
  "author": "Kristen1604",
  "sentiment": "positive",
  "summary": "The author expresses their love for Lift45 and mentions the difficulty in attending weekend classes due to the later timing.",
  "text": "I love them! Ive been wanting to do lifting at planet fitness but I never go. This is exactly what I need in my life.",
  "category": "exercise content"
} 

496 {
    "date": "2023-06-09",
    "topic": "Lift45",
    "title": "Re: Only doing Lift 45",
    "source": "Reddit",
    "author": "ladeeda923",
    "sentiment": "neutral",
    "summary": "The author expresses a wish for their studio to offer Lift45.",
    "text": "I wish my studio offered them",
    "category": "other"
} 

497 {
  "date": "2023-06-06",
  "topic": "Lift45",
  "title": "Lift45 TB2 - week of 6/5",
  "source": "Reddit",
  "author": "OTF_Aggie02",
  "sentiment": "positive",
  "summary": "The author enjoyed t

516 {
    "date": "2023-06-09",
    "topic": "Lift45",
    "title": "Re: Lift45 TB2 - week of 6/5",
    "source": "Reddit",
    "author": "Nsking83",
    "sentiment": "positive",
    "summary": "The author expresses excitement and anticipation for trying Lift45 before a 60-minute class.",
    "text": "Doing this one tomorrow before a 60!",
    "category": "exercise content"
} 

517 {
    "date": "2023-06-06",
    "topic": "Lift45",
    "title": "Re: Lift45 TB2 - week of 6/5",
    "source": "Reddit",
    "author": "nannertreeninja",
    "sentiment": "positive",
    "summary": "The author expresses excitement about booking a Lift45 class today.",
    "text": "Thanks for intel, just booked a class today",
    "category": "other"
} 

518 {
  "date": "2023-06-06",
  "topic": "Lift45",
  "title": "Lift45 TB3 - Week of 6/5",
  "source": "Reddit",
  "author": "lost-theory",
  "sentiment": "positive",
  "summary": "The post provides a detailed breakdown of the Lift45 workout routine and seems t

537 {
    "date": "2023-06-06",
    "topic": "Lift45",
    "title": "Re: Is there is a theme for LIFT45 templates week-to-week?",
    "source": "Reddit",
    "author": "civilcatlady",
    "sentiment": "positive",
    "summary": "The post expresses excitement and positivity about Lift45.",
    "text": "That would be awesome!!",
    "category": "other"
} 

538 {
    "date": "2023-06-07",
    "topic": "Lift45",
    "title": "Re: Is there is a theme for LIFT45 templates week-to-week?",
    "source": "Reddit",
    "author": "jenniferlynn5454",
    "sentiment": "positive",
    "summary": "The post expresses excitement and positive sentiment towards Lift45.",
    "text": "That would be EVERYTHING I'VE EVER WANTED!!!!!!",
    "category": "other"
} 

539 {
    "date": "2023-06-06",
    "topic": "Lift45",
    "title": "Re: Is there is a theme for LIFT45 templates week-to-week?",
    "source": "Reddit",
    "author": "Nsking83",
    "sentiment": "positive",
    "summary": "The author prefers the 

556 {
  "date": "2023-06-06",
  "topic": "Lift45",
  "title": "Re: Lift45 TB1 - week of 6/5",
  "source": "Reddit",
  "author": "oTfypical-Editor1561",
  "sentiment": "positive",
  "summary": "Nice class :)",
  "text": "Nice class :)",
  "category": "exercise content"
} 

557 {
    "date": "2023-06-06",
    "topic": "Lift45",
    "title": "Re: Lift45 TB1 - week of 6/5",
    "source": "Reddit",
    "author": "HHIOTF",
    "sentiment": "positive",
    "summary": "The author enjoyed the Lift45 template.",
    "text": "I really enjoyed this template",
    "category": "exercise content"
} 

558 {
    "date": "2023-06-06",
    "topic": "Lift45",
    "title": "Re: Lift45 TB1 - week of 6/5",
    "source": "Reddit",
    "author": "DogsandTitos",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment.",
    "text": "Thank you so much for posting!!",
    "category": "other"
} 

559 {
  "date": "2023-06-07",
  "topic": "Lift45",
  "title": "Re: Lift45 TB1 - week

### Converting the json_objects (Lift45 posts) into a DataFrame

In [143]:
lift45_df = pd.DataFrame(json_objects)

In [156]:
# Setting max column width to none so that the full summary and text could be seen
pd.set_option('display.max_colwidth', None)

In [152]:
lift45_df

,date,topic,title,source,author,sentiment,summary,text,category
0,2023-07-24,Lift45,Lift 45 TB1 Week of 7/24/23,Reddit,Crafty-Persimmon1857,positive,The author is seeking feedback on the exercises in Lift45 and expresses gratitude for any edits or corrections.,Help a sister out; some of these exercises I feel I put in the wrong block or mislabeled. Let me know what needs an edit. Thanks! Happy lifting. :),exercise content
1,2023-07-24,Lift45,Re: Lift 45 TB1 Week of 7/24/23,Reddit,Fickle-Huckleberry-8,positive,The post expresses positive customer sentiment and compliments about Lift45.,"I was just wishing for this sort of intel, thank you!!!! Looks awesome!!!!",other
2,2023-07-24,Lift45,Re: Lift 45 TB1 Week of 7/24/23,Reddit,backupjesus,positive,The author expresses frustration about missing their regular TB1 class but shows enthusiasm for trying Lift45 at another studio.,I missed my normal TB1 this morning due to a conflicting work call that the other person never showed up for. Grrr. This looks so good I may need to track down a TB1 at another local studio this week. Thanks for the intel OP!,course availability
3,2023-07-23,Lift45,Two different people borrowed from my station today. Lift45 format small rant.,Reddit,cindobeast,negative,"The author expresses frustration with the new format of Lift45 classes, where there are more participants and no designated floor sections, leading to the need to borrow weights and inconvenience.","Before in Lift45 classes we would all have a station and do Floor side first then Rower side exercises. Lately they've allowed more than 16 people to sign up so all floor stations had members plus like 4-5 people who start near the rowers. When we switch those 4-5 people take stations 1-5 and then those folks move to rower side of the gym. So when you don't have a designated floor section you have no choice but to ask people to borrow weights. We had 4 extra people but 2 of them borrowed from mine! They asked and we're nice and all that so it's not a rant against them we'll except one girl who borrowed 20s said she'd bring them back but never did lol. I was able to work with what I could find near me, but if I wanted to go 20s no one near me had them free anyway so it was just a little annoying and I blame it more of this new format than anything. Anyone else have this format at their lift?",course availability
4,2023-07-23,Lift45,Re: Two different people borrowed from my station today. Lift45 format small rant.,Reddit,Caballita14,negative,"The author expresses dissatisfaction with the size of the locations, the lack of weight racks, and the condition of the weight holders in Lift45.",They really need to get bigger locations and have an extra two weight rack. Also fix the weight holders so they arent death claws waiting to cut your fingers.,course availability
...,...,...,...,...,...,...,...,...,...
568,2023-06-05,Lift45,Re: Lift45 TB1 - week of 6/5,Reddit,ZeeKayYou,neutral,The post/comment is neutral and provides information about the lift templates for last week.,The lift templates go from Monday to Sunday. This was last weeks template.,other
569,2023-06-06,Lift45,Re: Lift45 TB1 - week of 6/5,Reddit,ctello1221,positive,The post/comment expresses appreciation and gratitude for the provided information about Lift45.,Oh thank you!! I didnt know that :) I appreciate the info!,other
570,2023-06-05,Lift45,Re: Lift45 TB1 - week of 6/5,Reddit,hotdogwater,positive,The author expresses regret for missing a Lift45 class last week but finds it appealing.,I'm sad I missed it last week! That looks so good though!,other
571,2023-06-05,Lift45,Re: Lift45 TB1 - week of 6/5,Reddit,April290,neutral,The post does not express a clear sentiment or provide specific feedback about Lift45.,This was what we did last week.,other


In [186]:
lift45_df['date'] = lift45_df['date'].str.replace('#', '')
lift45_df['date'] = pd.to_datetime(lift45_df['date'])

# Customer Sentiment about HR and Splats

## Fetching all Reddit posts, comments, and replies about HR Metrics from the last two days

In [217]:
two_days_ago = int(time.time()) - 2 * 24 * 60 * 60
flair = 'HR / Splats'

In [218]:
posts = []

for post in subreddit.new(limit = 1000):
    if post.link_flair_text == flair:
        if post.created_utc >= two_days_ago:
            decoded_text = codecs.unicode_escape_decode(post.selftext)[0]
            clean_text = decoded_text.replace('’', "'")
            clean_text = clean_text.encode('ascii', 'ignore').decode()
            
            posts.append({
                'date': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                'author': post.author.name if post.author else 'Deleted',
                'title': post.title,
                'body': clean_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
            })
            
            for comment in post.comments:
                fetch_comments(comment, posts)

In [219]:
# Combine all post bodies into one string
del all_splats_posts
all_splats_posts = " #END# ".join([post['body'] for post in posts])

In [220]:
all_splats_posts

'How do I burn more calories? I usually get between 375-400 calories per class. My splats are usually 13-20, and then green the rest of the time (well, maybe 3-4 minutes blue). Is there something I can do? #END# 1) ignore calories burned.  Its not that accurate.   2) youre supposed to spend most of the time in green with 12-20 splat points (orange and red).   It sounds like youre doing exactly what is expected for an OTF workout. #END# If you want to burn more calories, eat a little less. Like literally 5 bites is the difference here  The benefit of working out isnt and has never been calories burned #END# Without knowing anything about you, it sounds like youre probably right about where you should be. Im a 37F, 54 120lbs, in really good shape. At OTF, I spend around 15-20 min in the orange or red, and burn around 425 calories according to OTF (my Apple Watch reads in the high 300s). My fianc is 61 170 and burns about twice what I do. There is no way I could get close to his calorie b

### Top 5 positive and negative topics about splat points, max HR, and heart rate zones from the last 2 days

In [222]:
prompt = f"""
Your role is to analyze and summarize the main categories of customer sentiment and feedback about splat points, \
max HR, and heart rate zones that Orangetheory uses to track a member's workout.

Glossory of terms:
- Max HR, or max heart rate, is essentially your fitness level based on heart rate, \
and it changes based on the progress you are making at Orangetheory or in your health/fitness level in general.

- Heart rate zones. There are 5 heart rate zones: grey (resting), blue (easy), green (challenging), \
orange (uncomfortable), and red (All Out). 

- Splat points, or splats, are minutes in the orange or red heart rate zones. When your Max HR goes up, \
it makes it harder to achieve splat points.


You are presented with a compilation of these posts and comments, delimited by triple backticks. 

Analyze the text and identify the top topics that customers are positively and negatively commenting on. \
Focus only on the specific compliments, criticisms, and feedback about splat points, max HR, and \
heart rate zones. Ignore any unrelated comments. 

* Use at most 3 words as the topic name and give a brief explanation of each. 
Give me 5 positive and 5 negative topics.

Social Media Posts: ```{all_splats_posts}```
"""

response = get_completion(prompt)
print(response)

Positive topics:
1) Calorie burn: Customers are discussing ways to increase their calorie burn during Orangetheory workouts. They mention strategies such as strength-focused rowing, lifting heavier weights, and performing power moves on the floor. These strategies are seen as effective in helping them burn more calories.
2) Splat points: Customers mention earning splat points during their workouts, which indicates that they are spending time in the orange and red heart rate zones. They view earning splat points as a positive achievement and a sign of an effective workout.
3) Heart rate zones: Customers mention spending most of their time in the green heart rate zone, which is considered challenging. They see this as a positive indicator of their workout intensity and effort.
4) Fitness level: Customers discuss their fitness levels and how it affects their calorie burn. They mention that having more muscle mass can naturally increase calorie burn, even at rest. This is seen as a positiv

## Fetching all Reddit posts, comments, and replies about HR Metrics from the last three days

In [224]:
three_days_ago = int(time.time()) - 3 * 24 * 60 * 60
flair = 'HR / Splats'

In [225]:
posts = []

for post in subreddit.new(limit = 1000):
    if post.link_flair_text == flair:
        if post.created_utc >= three_days_ago:
            decoded_text = codecs.unicode_escape_decode(post.selftext)[0]
            clean_text = decoded_text.replace('’', "'")
            clean_text = clean_text.encode('ascii', 'ignore').decode()
            
            posts.append({
                'date': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                'author': post.author.name if post.author else 'Deleted',
                'title': post.title,
                'body': clean_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
            })
            
            for comment in post.comments:
                fetch_comments(comment, posts)

In [257]:
#remove the previous post about calorie to get more variety of feedback categories
three_d_posts = posts[15:60]

In [258]:
# Combine all post bodies into one string
three_d_posts_str = " #END# ".join([post['body'] for post in three_d_posts])

In [259]:
three_d_posts_str

'2 questions: I feel like Im killing myself, but Im staying in the green way too much.  I know that they mess with the heart rate monitors, but is that the only way to get back into the Orange or is it actually more beneficial to stay in the green? For example the last class I was 41 minutes in the green 10 minutes in orange, and the rest was blue and gray.  Q2 I look up the class plan and I kind of get over excited and I feel like when I start I have this adrenaline rush and it very quickly leaves and I dont do as well as I would think I would do? I like to know ahead of time because sometimes it helps me figure out if its a rest day or not and I was just trying to figure out if Im in a plateau and if so what to do to make it different. #END# Someone will likely articulate this better but you are likely improving your conditioning to the point that you are staying in the green even with your increased efforts which is not a problem. I rarely get the 12 splats anymore and stay in the g

In [260]:
prompt = f"""
Your role is to analyze and summarize the main categories of customer sentiment and feedback about splat points, \
max HR, and heart rate zones that Orangetheory uses to track a member's workout.

Glossory of terms:
- Max HR, or max heart rate, is essentially your fitness level based on heart rate, \
and it changes based on the progress you are making at Orangetheory or in your health/fitness level in general.

- Heart rate zones. There are 5 heart rate zones: grey (resting), blue (easy), green (challenging), \
orange (uncomfortable), and red (All Out). 

- Splat points, or splats, are minutes in the orange or red heart rate zones. When your Max HR goes up, \
it makes it harder to achieve splat points.


You are presented with a compilation of these posts and comments, delimited by triple backticks. 

Analyze the text and identify the top topics that customers are positively and negatively commenting on. \
Focus only on the specific compliments, criticisms, and feedback about splat points, max HR, and \
heart rate zones. Ignore any unrelated comments. 

* Use at most 3 words as the topic name and give a brief explanation of each. 
Give me 5 positive and 5 negative topics.

Social Media Posts: ```{three_d_posts_str}```
"""

response = get_completion(prompt)
print(response)

Positive topics:
1. Improved Conditioning: Customers are positively commenting on their ability to stay in the green heart rate zone for longer periods of time, indicating improved conditioning and fitness levels.
2. Recovery Rate: Customers are pleased with their quick recovery rates, dropping back into the green heart rate zone soon after completing a challenging block or exercise.
3. Excitement about Exercise: Customers express excitement and enthusiasm about their workouts, emphasizing that the number of splat points does not determine the benefits of exercising.
4. Starting on Treadmill: Customers find that starting on the treadmill helps them achieve more splat points and sustain higher heart rates throughout the class.
5. Personalized Heart Rate Adjustments: Customers appreciate that Orangetheory adjusts their heart rate zones based on individual factors, allowing them to achieve more splat points and have a more accurate representation of their effort level.

Negative topics:
1

### 5 overarching categories of feedback of splat points, max HR, and heart rate zones

1. **Workout Effectiveness and Personal Progress**: This overarching topic encapsulates discussions on customers' personal fitness journey, their progress, fitness levels, and how effectively they are able to achieve their workout goals.

2. **Performance Metrics and Tracking**: This broad topic covers discussions and feedback related to various performance metrics, including splat points, max HR, heart rate zones, and calorie burn, and the accuracy or perceived limitations of these tracking systems.

3. **Workout Strategies and Preferences**: This includes various workout techniques, strategies, and workout starting preferences (like treadmill or floor first), which can affect the workout's effectiveness and personal achievements measured by splat points.

4. **Age and Physiological Factors**: This topic focuses on feedback about how innate characteristics like age, gender, weight, long-term health conditions, and other physiological factors can influence workout outcomes and performance metrics like splat points.

5. **Impact of External Factors**: This topic captures discussions about how factors external to the workout, like asthma medication, environmental conditions, or other temporary health conditions, can influence workout performance and tracking metrics.

## Fetching all Reddit posts, comments, and replies about splat points, max HR, and heart rate zones from the last 3 months

In [262]:
three_months_ago = int(time.time()) - 90 * 24 * 60 * 60
flair = 'HR / Splats'

In [263]:
posts = []

for post in subreddit.new(limit = 1000):
    if post.link_flair_text == flair:
        if post.created_utc >= three_months_ago:
            decoded_text = codecs.unicode_escape_decode(post.selftext)[0]
            clean_text = decoded_text.replace('’', "'")
            clean_text = clean_text.encode('ascii', 'ignore').decode()
            
            posts.append({
                'date': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                'author': post.author.name if post.author else 'Deleted',
                'title': post.title,
                'body': clean_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
            })
            
            for comment in post.comments:
                fetch_comments(comment, posts)

/var/folders/nl/4h1npm454flgnps4vlz6y_700000gn/T/ipykernel_54082/3147963917.py:3: DeprecationWarning: invalid escape sequence '\*'
  decoded_comment = codecs.unicode_escape_decode(comment.body)[0]


In [264]:
three_month_splat = posts

## Writing a prompt for creating a list of posts and comments that are about customer sentiment of splat points, max HR, and heart rate zones

### Using the first 20 posts and comments for testing the prompt

In [265]:
test_posts = three_month_splat[:20]

In [266]:
test_posts

[{'date': '2023-07-23',
  'author': 'Inside_Beginning_740',
  'title': 'Calories',
  'body': 'How do I burn more calories? I usually get between 375-400 calories per class. My splats are usually 13-20, and then green the rest of the time (well, maybe 3-4 minutes blue). Is there something I can do?'},
 {'date': '2023-07-23',
  'author': 'pantherluna',
  'title': 'Re: Calories',
  'body': '1) ignore calories burned.  Its not that accurate.   2) youre supposed to spend most of the time in green with 12-20 splat points (orange and red).   It sounds like youre doing exactly what is expected for an OTF workout.'},
 {'date': '2023-07-24',
  'author': 'bonniejo514',
  'title': 'Re: Calories',
  'body': 'If you want to burn more calories, eat a little less. Like literally 5 bites is the difference here  The benefit of working out isnt and has never been calories burned'},
 {'date': '2023-07-23',
  'author': 'TopOfTheCurve',
  'title': 'Re: Calories',
  'body': 'Without knowing anything about yo

### Prompt Template for splat points, max HR, and heart rate zones on Reddit

In [279]:
for i in range(len(test_posts)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    Orangetheory's biometric way of measuring a member's workout performance, including using splat points, \
    max HR, and heart rate zones. Orangetheory a fitness company that is sometimes referred to as OTF. 
    
    Glossory of terms:
        - Max HR, or max heart rate, is essentially your fitness level based on heart rate, \
        and it changes based on the progress you are making at Orangetheory or in your health/fitness level \
        in general.

        - Heart rate zones. There are 5 heart rate zones: grey (resting), blue (easy), green (challenging), \
        orange (uncomfortable), and red (All Out). The heart rate zones change based on your max HR.

        - Splat points, or splats, are minutes in the orange or red heart rate zones. When your Max HR goes up, \
        it makes it harder to achieve splat points.
    

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{test_posts[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{test_posts[i]['date']}#
    
    2. topic: 'HR and splats'

    3. title: ##{test_posts[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{test_posts[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about splat points or splats, max HR, heart rate zones, calorie burn, and \
    other fitness tracking metrics.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about splat points or splats, max HR, heart rate zones, calorie burn, and \
    other fitness tracking metrics. Do not extract any other texts.   

    9. category: 
    
        6 categories of feedback are as follows:

        - workout progress: This overarching topic encapsulates discussions on \
        customers' personal fitness journey, their progress, and fitness levels. 

        - metric tracking: This broad topic covers discussions and feedback related to the accuracy or perceived \
        limitations of the tracking systems for splat points, max HR, heart rate zones, and calorie burn.

        - workout strategies: This includes various workout techniques, strategies, \
        and workout starting preferences (like treadmill or floor first), which can affect how many \
        splat points a member earns and how many calories a member burns.

        - physiological factors: This topic focuses on feedback about how innate characteristics \
        like age, gender, weight, long-term health conditions, and other physiological factors can influence \
        workout outcomes and performance metrics like splat points and calories burned.

        - external influences: This topic captures discussions about how factors external to the workout \
        such as medication and the environment can influence workout performance and tracking metrics.
        
        - fitness knowledge: This category encompasses understanding of fitness metrics and principles.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'workout progress', 'metric tracking', 'workout strategies', 'physiological factors', \
    'external influences', and 'fitness knowledge'.
    Use "other" if the post/comment does not fall into any of the 6 categories. 
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i, response, "\n")
    
#     # Load the response string as a dictionary and add to the list
#     json_objects.append(json.loads(response))

0 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Calories",
    "source": "Reddit",
    "author": "Inside_Beginning_740",
    "sentiment": "neutral",
    "summary": "The author is seeking advice on how to burn more calories during Orangetheory workouts, mentioning their current calorie burn, splat points, and heart rate zones.",
    "text": "How do I burn more calories? I usually get between 375-400 calories per class. My splats are usually 13-20, and then green the rest of the time (well, maybe 3-4 minutes blue). Is there something I can do?",
    "category": "workout strategies"
} 

1 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Re: Calories",
    "source": "Reddit",
    "author": "pantherluna",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment and provides feedback about splat points and heart rate zones, stating that spending most of the time in the green zone with 12-20 splat points is exp

11 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Re: Calories",
    "source": "Reddit",
    "author": "Inside_Beginning_740",
    "sentiment": "neutral",
    "summary": "The post is discussing TDEE (Total Daily Energy Expenditure) and does not provide specific feedback or sentiment about splat points, max HR, heart rate zones, or other fitness tracking metrics.",
    "text": "TDEE??",
    "category": "other"
} 

12 {
  "date": "2023-07-23",
  "topic": "HR and splats",
  "title": "Re: Calories",
  "source": "Reddit",
  "author": "thekathied",
  "sentiment": "neutral",
  "summary": "The post is a neutral response discussing total daily energy expenditure and burning more calories outside of Orangetheory Fitness (OTF).",
  "text": "Total daily energy expenditure. So to translate, this helpful response said: \"burn more calories outside of otf by burning more calories in the day.\" Edit: the order.",
  "category": "other"
} 

13 {
    "date": "2023-07-23",
    "to

### Running the prompt with posts, comments, replies about HR/Splat from Reddit

In [281]:
#how many posts, comments, replies about Lift45 before adding HR/Splat ones
len(json_objects)

573

In [282]:
len(three_month_splat)

608

#### First 101 posts

In [285]:
cut_three_month_splat = three_month_splat[:101]

In [294]:
def run_splat_prompt():
    for i in range(len(cut_three_month_splat)):
        prompt = f"""
        Your role is to summarize relevant customer sentiment information from a social media post or comment about \
        Orangetheory's biometric way of measuring a member's workout performance, including using splat points, \
        max HR, and heart rate zones. Orangetheory a fitness company that is sometimes referred to as OTF. 

        Glossory of terms:
            - Max HR, or max heart rate, is essentially your fitness level based on heart rate, \
            and it changes based on the progress you are making at Orangetheory or in your health/fitness level \
            in general.

            - Heart rate zones. There are 5 heart rate zones: grey (resting), blue (easy), green (challenging), \
            orange (uncomfortable), and red (All Out). The heart rate zones change based on your max HR.

            - Splat points, or splats, are minutes in the orange or red heart rate zones. When your Max HR goes up, \
            it makes it harder to achieve splat points.


        The social media post or comment of interest is delimited by 3 backticks. 
        Social Media Post: ```{cut_three_month_splat[i]['body']}```

        The below 10 fields are the JSON keys that you should output after analyzing the post. 

        1. date: #{cut_three_month_splat[i]['date']}#

        2. topic: 'HR and splats'

        3. title: ##{cut_three_month_splat[i]['title']}##

        4. source: 'Reddit'

        5. author: ###{cut_three_month_splat[i]['author']}###

        6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
        Use 1 of the 3 words only.

        7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
        criticisms, and feedback about splat points or splats, max HR, heart rate zones, calorie burn, and \
        other fitness tracking metrics.

        8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
        and feedback about splat points or splats, max HR, heart rate zones, calorie burn, and \
        other fitness tracking metrics. Do not extract any other texts.   

        9. category: 

            6 categories of feedback are as follows:

            - workout progress: This overarching topic encapsulates discussions on \
            customers' personal fitness journey, their progress, and fitness levels. 

            - metric tracking: This broad topic covers discussions and feedback related to the accuracy or perceived \
            limitations of the tracking systems for splat points, max HR, heart rate zones, and calorie burn.

            - workout strategies: This includes various workout techniques, strategies, \
            and workout starting preferences (like treadmill or floor first), which can affect how many \
            splat points a member earns and how many calories a member burns.

            - physiological factors: This topic focuses on feedback about how innate characteristics \
            like age, gender, weight, long-term health conditions, and other physiological factors can influence \
            workout outcomes and performance metrics like splat points and calories burned.

            - external influences: This topic captures discussions about how factors external to the workout \
            such as medication and the environment can influence workout performance and tracking metrics.

            - fitness knowledge: This category encompasses understanding of fitness metrics and principles.

        Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
        about, including 'workout progress', 'metric tracking', 'workout strategies', 'physiological factors', \
        'external influences', and 'fitness knowledge'.
        Use "other" if the post/comment does not fall into any of the 6 categories. 
        Do not leave blank.

        ----------
        YOUR TASK: 
        Analyze the post for each of the JSON keys mentioned above and \
        output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.

        """

        response = get_completion(prompt)
        print(i, response, "\n")

        # Load the response string as a dictionary and add to the list
        json_objects.append(json.loads(response))

In [286]:
run_splat_prompt()

0 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Calories",
    "source": "Reddit",
    "author": "Inside_Beginning_740",
    "sentiment": "neutral",
    "summary": "The author is seeking advice on how to burn more calories during Orangetheory workouts, mentioning their current calorie burn, splat points, and heart rate zones.",
    "text": "How do I burn more calories? I usually get between 375-400 calories per class. My splats are usually 13-20, and then green the rest of the time (well, maybe 3-4 minutes blue). Is there something I can do?",
    "category": "workout strategies"
} 

1 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Re: Calories",
    "source": "Reddit",
    "author": "pantherluna",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment and provides feedback about splat points and heart rate zones, stating that spending most of the time in the green zone with 12-20 splat points is exp

12 {
  "date": "2023-07-23",
  "topic": "HR and splats",
  "title": "Re: Calories",
  "source": "Reddit",
  "author": "thekathied",
  "sentiment": "neutral",
  "summary": "The post is discussing total daily energy expenditure and how to burn more calories outside of Orangetheory Fitness (OTF).",
  "text": "Total daily energy expenditure. So to translate, this helpful response said: \"burn more calories outside of otf by burning more calories in the day.\" Edit: the order.",
  "category": "other"
} 

13 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Re: Calories",
    "source": "Reddit",
    "author": "HolyMackerelTabby",
    "sentiment": "neutral",
    "summary": "The post/comment does not provide any specific feedback or sentiment about splat points, max HR, heart rate zones, or other fitness tracking metrics.",
    "text": "",
    "category": "other"
} 

14 {
    "date": "2023-07-24",
    "topic": "HR and splats",
    "title": "Re: Calories",
    "source": "R

23 {
  "date": "2023-07-22",
  "topic": "HR and splats",
  "title": "Re: Way too much green despite feeling working hard/ over excitement and then don’t do well in class",
  "source": "Reddit",
  "author": "spottysasquatch",
  "sentiment": "positive",
  "summary": "The post expresses excitement and satisfaction with the increase in endurance, strength, and stamina, resulting in spending more time in the green heart rate zone. The author suggests increasing intensity to spend more time in the orange zone.",
  "text": "First, keep in mind that green is still the fat burning zone, so although youre not hitting as much after burn youre still getting a great workout even in the green. That said, Ive noticed Im spending much more time in the green now because my endurance, strength, and stamina have all increased dramatically in the three months since I first joined. I *never* used to get back down to the green during WRs on the treadmill when I first started, now I easily get down to the gr

30 {
    "date": "2023-07-22",
    "topic": "HR and splats",
    "title": "Re: Way too much green despite feeling working hard/over excitement and then don’t do well in class",
    "source": "Reddit",
    "author": "avprobeauty",
    "sentiment": "neutral",
    "summary": "The author agrees with the sentiment expressed but refrains from commenting further to avoid confusion.",
    "text": "I'm pretty analytical and I agree with you. I am not going to comment because I think i might confuse more by accident haha",
    "category": "other"
} 

31 {
    "date": "2023-07-22",
    "topic": "HR and splats",
    "title": "Re: Way too much green despite feeling working hard/over excitement and then don’t do well in class",
    "source": "Reddit",
    "author": "Maurag12",
    "sentiment": "negative",
    "summary": "The author expresses frustration about experiencing too much time in the green heart rate zone despite feeling like they are working hard, leading to poor performance in class.",
  

44 {
    "date": "2023-07-22",
    "topic": "HR and splats",
    "title": "Re: Asthmatics & OTF",
    "source": "Reddit",
    "author": "Otffollow",
    "sentiment": "neutral",
    "summary": "The post does not provide any specific feedback or sentiment about splat points, max HR, heart rate zones, calorie burn, or other fitness tracking metrics.",
    "text": "This, or very strong mint gum while on the treadmill.",
    "category": "other"
} 

45 {
    "date": "2023-07-22",
    "topic": "HR and splats",
    "title": "Re: Asthmatics & OTF",
    "source": "Reddit",
    "author": "pahelisolved",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment and appreciation for the tip.",
    "text": "Wow what a good tip. I am definitely giving it a try. Thanks!",
    "category": "other"
} 

46 {
    "date": "2023-07-23",
    "topic": "HR and splats",
    "title": "Re: Asthmatics & OTF",
    "source": "Reddit",
    "author": "Soccermom256",
    "sentiment": "ne

58 {
    "date": "2023-07-22",
    "topic": "HR and splats",
    "title": "Re: Asthmatics & OTF",
    "source": "Reddit",
    "author": "Otffollow",
    "sentiment": "neutral",
    "summary": "The post discusses the author's experience with their heart rate being high after using an inhaler before class, and they are unsure if it was due to the medication or their pre-existing condition.",
    "text": "A few times I've done it in the lobby right before class and my heart rate was high. Not sure if it was the albuterol or that I was already feeling tight that day.",
    "category": "external influences"
} 

59 {
    "date": "2023-07-22",
    "topic": "HR and splats",
    "title": "Re: Asthmatics & OTF",
    "source": "Reddit",
    "author": "befk",
    "sentiment": "neutral",
    "summary": "The post discusses the increased heart rate as a side effect of medication, but does not provide any specific feedback or sentiment about splat points, max HR, heart rate zones, or other fitness tra

71 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Beta Blockers",
    "source": "Reddit",
    "author": "notdominique",
    "sentiment": "neutral",
    "summary": "The post discusses the importance of taking medication consistently for consistent results, without providing specific feedback or opinions about splat points, max HR, heart rate zones, or other fitness tracking metrics.",
    "text": "Take your meds at the same time every day. That's really the only way you're going to have consistent results. Do the behavior consistently.",
    "category": "external influences"
} 

72 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Beta Blockers",
    "source": "Reddit",
    "author": "sallyweld",
    "sentiment": "positive",
    "summary": "The author mentions taking beta blockers and how their heart rate zones do not necessarily match their effort, but they still find the workout beneficial for their heart health.",
    "text": "I 

83 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Zero splat point",
    "source": "Reddit",
    "author": "No_Star_9327",
    "sentiment": "neutral",
    "summary": "The post/comment mentions that heart-rate calibration gets personalized after 20 classes, providing information about the process of personalizing heart rate zones and suggesting a neutral sentiment towards this feature.",
    "text": "For clarification, heart-rate calibration gets personalized after 20 classes.",
    "category": "metric tracking"
} 

84 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Zero splat point",
    "source": "Reddit",
    "author": "FortuneDesigner",
    "sentiment": "neutral",
    "summary": "The author is seeking clarification on the meaning of splat points and their relationship to max HR and heart rate zones.",
    "text": "Can you expand on what you mean here?",
    "category": "fitness knowledge"
} 

85 {
    "date": "2023-07-22",
   

95 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Zero splat point",
    "source": "Reddit",
    "author": "Why_So-Serious",
    "sentiment": "neutral",
    "summary": "The post discusses the discrepancy between the heart rate shown on the author's watch and the OT monitor, suggesting that wearing a watch with a heart rate monitor can provide more accurate data.",
    "text": "If your heart rate is actually up. You will get splats. Do you have a watch with heart rate monitor? Does it show youre in the orange zone but the OT monitor isnt showing that? I wear a watch with monitor at class and can see my OT data in a different app.",
    "category": "metric tracking"
} 

96 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Zero splat point",
    "source": "Reddit",
    "author": "Ed_the_chosen_one",
    "sentiment": "positive",
    "summary": "The author agrees and mentions that their pixel watch is within 1-2 bpm of Orangetheory's bi

#### Next 100 posts

In [287]:
cut_three_month_splat = three_month_splat[101:201]

In [289]:
run_splat_prompt()

0 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Zero splat point",
    "source": "Reddit",
    "author": "geraltthedragon",
    "sentiment": "negative",
    "summary": "The post expresses a negative sentiment regarding the heart rate monitor's accuracy in reading heart rate due to sweat drying on the sensor.",
    "text": "Sometimes sweat dries on the sensor in the monitor and doesn't read the heart rate properly. It's definitely happened to me a couple of times in the last couple of months. Try wiping down the sensor before each class to reduce this.",
    "category": "metric tracking"
} 

1 {
    "date": "2023-07-21",
    "topic": "HR and splats",
    "title": "Re: Zero splat point",
    "source": "Reddit",
    "author": "m1dnite",
    "sentiment": "neutral",
    "summary": "The post is about a user experiencing issues with their armband not picking up anything on the rower, indicating a neutral sentiment.",
    "text": "Yeah I had to readjust my armband

13 {
    "date": "2023-07-20",
    "topic": "HR and splats",
    "title": "Re: Earning splats",
    "source": "Reddit",
    "author": "TBGH_96",
    "sentiment": "positive",
    "summary": "The post expresses gratitude and satisfaction with the splat points system.",
    "text": "TY!",
    "category": "other"
} 

14 {
    "date": "2023-07-20",
    "topic": "HR and splats",
    "title": "Re: Earning splats",
    "source": "Reddit",
    "author": "jenniferlynn5454",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive customer sentiment about earning splat points and discusses the impact of max HR on achieving splats.",
    "text": "When your Max HR goes up, it makes it harder to achieve splat points.",
    "category": "metric tracking"
} 

15 {
    "date": "2023-07-20",
    "topic": "HR and splats",
    "title": "Re: Earning splats",
    "source": "Reddit",
    "author": "realistnotsorry",
    "sentiment": "neutral",
    "summary": "The post is seeking clarif

28 {
    "date": "2023-07-19",
    "topic": "HR and splats",
    "title": "Re: No red zone, barely any points",
    "source": "Reddit",
    "author": "Worth-Locksmith-127",
    "sentiment": "neutral",
    "summary": "The post/comment expresses a neutral sentiment and briefly mentions checking something, but it is not clear what it is referring to.",
    "text": "Yes. I just checked. Good idea though!!!",
    "category": "other"
} 

29 {
  "date": "2023-07-19",
  "topic": "HR and splats",
  "title": "Re: No red zone, barely any points",
  "source": "Reddit",
  "author": "brad0022",
  "sentiment": "neutral",
  "summary": "The author updated their biometric measurements and expected it to auto-update based on inbody scan results.",
  "text": "Thanks. I just updated mine. I assumed it would auto-update based on the inbody scan results.",
  "category": "metric tracking"
} 

30 {
  "date": "2023-07-19",
  "topic": "HR and splats",
  "title": "Re: No red zone, barely any points",
  "source": 

42 {
    "date": "2023-07-19",
    "topic": "HR and splats",
    "title": "Re: No red zone, barely any points",
    "source": "Reddit",
    "author": "MaximumUsual880",
    "sentiment": "negative",
    "summary": "The author expresses dissatisfaction with not being able to reach the red zone and earn enough splat points.",
    "text": "No red zone, barely any points",
    "category": "metric tracking"
} 

43 {
  "date": "2023-07-19",
  "topic": "HR and splats",
  "title": "Re: No red zone, barely any points",
  "source": "Reddit",
  "author": "Worth-Locksmith-127",
  "sentiment": "neutral",
  "summary": "The author expresses their concern about not seeing a significant change in their body despite regularly attending Orangetheory workouts and taking breaks. They mention focusing on better eating habits and attribute the lack of progress to their age.",
  "text": "I dont feel like I see a huge change in my body. It might be my pesky age.",
  "category": "workout progress"
} 

44 {
  "da

55 {
    "date": "2023-07-13",
    "topic": "HR and splats",
    "title": "Re: Am I doing something wrong?",
    "source": "Reddit",
    "author": "UofHCoog",
    "sentiment": "neutral",
    "summary": "The post is questioning if the low calorie count indicates an issue with the heart monitor, and asks about spending most minutes in the green heart rate zone.",
    "text": "If it is a heart monitor issue, then you should be seeing lots of grey. With calorie count of 800-900, it doesn't sound like it? Are most of your minutes spent in the green?",
    "category": "metric tracking"
} 

56 {
    "date": "2023-07-13",
    "topic": "HR and splats",
    "title": "Re: Am I doing something wrong?",
    "source": "Reddit",
    "author": "loganrb",
    "sentiment": "neutral",
    "summary": "The post/comment states that most people are in the green heart rate zone, indicating a neutral sentiment towards the workout performance measurement at Orangetheory.",
    "text": "Yes most are in green",
 

68 {
    "date": "2023-07-13",
    "topic": "HR and splats",
    "title": "Re: Am I doing something wrong?",
    "source": "Reddit",
    "author": "Realistic_Warthog_23",
    "sentiment": "neutral",
    "summary": "The author is seeking clarification on whether wearing the heart rate monitor on the forearm affects the accuracy of the measurements.",
    "text": "I don't know if this makes a difference but I wear the monitor on my forearm (just below elbow) not bicep.",
    "category": "metric tracking"
} 

69 {
    "date": "2023-07-13",
    "topic": "HR and splats",
    "title": "Re: Am I doing something wrong?",
    "source": "Reddit",
    "author": "loganrb",
    "sentiment": "neutral",
    "summary": "The post expresses a neutral sentiment and indicates the author's intention to try out Orangetheory's biometric way of measuring workout performance.",
    "text": "Will try it out",
    "category": "other"
} 

70 {
    "date": "2023-07-14",
    "topic": "HR and splats",
    "title": "

81 {
  "date": "2023-07-11",
  "topic": "HR and splats",
  "title": "Re: HR lower in morning classes compared to evening?",
  "source": "Reddit",
  "author": "slaine1077",
  "sentiment": "neutral",
  "summary": "The post discusses various factors that can affect heart rate during workouts, including caffeine, salt, food, hydration, hormones, and the timing of consumption. No specific feedback or sentiment about splat points, max HR, heart rate zones, or other fitness tracking metrics is provided.",
  "text": "Caffeine, salt, food, hydration. Pay attention to what youre consuming last at night and how long it is before you consume things the next day and then what you consume before an afternoon/evening class. Also, if youre female, then hormones can affect it too. So many factors.",
  "category": "other"
} 

82 {
    "date": "2023-07-11",
    "topic": "HR and splats",
    "title": "Re: HR lower in morning classes compared to evening?",
    "source": "Reddit",
    "author": "pattyd2828"

93 {
    "date": "2023-07-07",
    "topic": "HR and splats",
    "title": "Re: Failing to hit splats after a long break",
    "source": "Reddit",
    "author": "KinvaraSarinth",
    "sentiment": "neutral",
    "summary": "The post/comment states that weight has no bearing on HR and splats, only affecting the calorie estimate.",
    "text": "Weight has no bearing on HR and splats. It will only affect the calorie estimate.",
    "category": "metric tracking"
} 

94 {
    "date": "2023-07-07",
    "topic": "HR and splats",
    "title": "Re: Failing to hit splats after a long break",
    "source": "Reddit",
    "author": "splat_bot",
    "sentiment": "neutral",
    "summary": "The post discusses the topics of heart rate zones and splat points, with links provided for discussions on not enough splat points, too many splat points, difficulty getting into the orange zone, and always being in the red zone.",
    "text": "Heart rate zones and splat points are very popular topics on this sub. Pl

#### Next 100 posts

In [290]:
cut_three_month_splat = three_month_splat[201:301]

In [291]:
run_splat_prompt()

0 {
    "date": "2023-07-07",
    "topic": "HR and splats",
    "title": "Re: Updates to heart-rate zone?",
    "source": "Reddit",
    "author": "LaurentiaF",
    "sentiment": "positive",
    "summary": "The author expresses excitement about their increased max heart rate and seeks clarification on whether it is a positive development.",
    "text": "Max heart rate went up by 2 beats! Not sure if thats a good thing",
    "category": "workout progress"
} 

1 {
  "date": "2023-07-07",
  "topic": "HR and splats",
  "title": "Re: Updates to heart-rate zone?",
  "source": "Reddit",
  "author": "_j_cow_82",
  "sentiment": "neutral",
  "summary": "The post is asking about the approximate date for updates to heart-rate zones, indicating a neutral sentiment.",
  "text": "Do you see an approx date though when it does update? I guess I can pay attention to when it does.",
  "category": "other"
} 

2 {
    "date": "2023-07-07",
    "topic": "HR and splats",
    "title": "Re: Updates to heart-rate

16 {
    "date": "2023-07-07",
    "topic": "HR and splats",
    "title": "Re: Cals vs. splat points?",
    "source": "Reddit",
    "author": "kjh3030",
    "sentiment": "neutral",
    "summary": "The post acknowledges that men usually weigh more and have higher muscle percentage, which factors into calories burned.",
    "text": "Men usually weigh more and have higher muscle % which factor into calories burned.",
    "category": "physiological factors"
} 

17 {
  "date": "2023-07-06",
  "topic": "HR and splats",
  "title": "Re: Cals vs. splat points?",
  "source": "Reddit",
  "author": "TopOfTheCurve",
  "sentiment": "neutral",
  "summary": "The post provides advice on not comparing calorie burn with others, emphasizes the importance of paying attention to actual heart rate rather than splat points, and suggests working in the lower end of the orange zone for more splats.",
  "text": "Theres a few factors at play, but my biggest advice is seriously dont worry about what others are bur

25 {
    "date": "2023-07-06",
    "topic": "HR and splats",
    "title": "Re: Cals vs. splat points?",
    "source": "Reddit",
    "author": "Otherwise_Nature_506",
    "sentiment": "neutral",
    "summary": "The author mentions that they tend to hover around 83% HR and may not get many splat points, but they can still burn the same calories. They are unsure if this is a good explanation and are seeking input from others.",
    "text": "I tend sometimes to hover around 83% HR so I may not get many splat points but I can still burn the same calories as I do when I get more splat points. I really notice this on days where Ive deliberately set a goal for a green day. Not sure if thats a good explanation or not  Hopefully others will chime in.",
    "category": "metric tracking"
} 

26 {
    "date": "2023-07-06",
    "topic": "HR and splats",
    "title": "Re: Cals vs. splat points?",
    "source": "Reddit",
    "author": "TopOfTheCurve",
    "sentiment": "neutral",
    "summary": "The po

36 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "AllTheCatsNPlants",
    "sentiment": "negative",
    "summary": "The author expresses anxiety and dissatisfaction with the statistics provided by Orangetheory's biometric way of measuring workout performance, leading them to stop wearing the monitor and instead listen to their body for better workouts.",
    "text": "The statistics gave me a lot of anxiety, so I stopped wearing the monitor. My workouts are better than ever because Im listening to my body instead of focusing on data.",
    "category": "metric tracking"
} 

37 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "Dontgeauxyet",
    "sentiment": "neutral",
    "summary": "The post is neutral and does not provide any specific feedback or sentiment about splat points, max HR, heart rate zones, or ot

49 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "Senior_Awareness_464",
    "sentiment": "negative",
    "summary": "The author expresses a negative sentiment, suggesting that the high number of splat points they receive indicates they are out of shape.",
    "text": "Thats funny. I think the fact that I get so many means Im out of shape!",
    "category": "workout progress"
} 

50 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "squatter_",
    "sentiment": "negative",
    "summary": "The post criticizes Orangetheory's estimation of max heart rate, suggesting that if the percentage on the board exceeds 100%, it indicates an inaccurate estimation. The author also mentions that the recalibration should occur after 20 classes.",
    "text": "Does the percentage on the board ever exceed 100%? Like does it r

62 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "pondertart",
    "sentiment": "positive",
    "summary": "The author had a positive experience with splat points, achieving over 50 after returning from a pandemic hiatus, and mentions that the heart rate monitor (HRM) adjusts along with the body's progress.",
    "text": "I think I had 56 (!!) splat points my first class, and over 50 for sure when I went back after the pandemic hiatus. As other folks are saying, your HRM will adjust (and so will your body once it realizes you really aren't trying to kill it!)",
    "category": "workout progress"
} 

63 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "JenEmm76",
    "sentiment": "positive",
    "summary": "The author has been attending Orangetheory Fitness (OTF) for 7 months and has seen improvements in th

73 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "smstrese",
    "sentiment": "neutral",
    "summary": "The post discusses the challenge of consistently achieving under 30 splat points in Orangetheory classes, emphasizing the importance of individual differences and focusing on recovery and injury prevention.",
    "text": "Regardless of HR calibration, it took me at least 6 months if not longer to consistently be getting under 30 splits a class. Everyone's body is different and OTF is challenging, especially if you're someone who always goes hard. Don't Worry too much, focus on your recovery times and not getting injured.",
    "category": "workout progress"
} 

74 {
    "date": "2023-07-05",
    "topic": "HR and splats",
    "title": "Re: Too many splat points?",
    "source": "Reddit",
    "author": "Low-Relationship9488",
    "sentiment": "neutral",
    "summary": "The author mentions that

85 {
    "date": "2023-07-03",
    "topic": "HR and splats",
    "title": "Re: When will my max hr update?",
    "source": "Reddit",
    "author": "OkContest5417",
    "sentiment": "positive",
    "summary": "The post/comment expresses a positive sentiment about the ability to update high heart rate data and bring it back to normal range.",
    "text": "You can message app support and ask them to remove the high heart rate from that date that's skewing your HR average. It took 24 hours, but they updated mine and it brought me back in my normal range. Mine was way off due to a change in beta blocker dosage.",
    "category": "metric tracking"
} 

86 {
    "date": "2023-07-04",
    "topic": "HR and splats",
    "title": "Re: When will my max hr update?",
    "source": "Reddit",
    "author": "asirenoftitan",
    "sentiment": "positive",
    "summary": "The post expresses gratitude and does not provide any specific feedback or criticism about splat points, max HR, heart rate zones, calori

99 {
    "date": "2023-07-03",
    "topic": "HR and splats",
    "title": "Re: Splats?",
    "source": "Reddit",
    "author": "Cook42523",
    "sentiment": "neutral",
    "summary": "The post/comment discusses the concept of splat points and emphasizes the importance of spending time in the orange and red heart rate zones, while cautioning against staying in the red zone for too long.",
    "text": "Just a reminder splat points are completely made up and if you went to a cardiologist and asked them a question about your splat points theyd look at you with concern. AppDude27 is right though its how long youre in the orange and red zone which is above 75% of your max heart rate. You dont want to be in the red for too long.",
    "category": "metric tracking"
} 



#### Next 100 posts

In [292]:
cut_three_month_splat = three_month_splat[301:401]

In [293]:
run_splat_prompt()

0 {
    "date": "2023-07-03",
    "topic": "HR and splats",
    "title": "Re: Splats?",
    "source": "Reddit",
    "author": "This_Beat2227",
    "sentiment": "neutral",
    "summary": "The post/comment provides feedback on the heart rate zones, specifically mentioning that orange is 84-91% of max HR instead of 75%.",
    "text": "Orange is 84-91% of max HR (not 75%).",
    "category": "metric tracking"
} 

1 {
    "date": "2023-07-04",
    "topic": "HR and splats",
    "title": "Re: Splats?",
    "source": "Reddit",
    "author": "Cook42523",
    "sentiment": "neutral",
    "summary": "The post is neutral and acknowledges a mistake made by the author.",
    "text": "Hahaha yes youre right! My mistake thank you!",
    "category": "other"
} 

2 {
    "date": "2023-07-03",
    "topic": "HR and splats",
    "title": "Re: Splats?",
    "source": "Reddit",
    "author": "OkRegular167",
    "sentiment": "neutral",
    "summary": "The author expresses a neutral sentiment towards splat points

12 {
  "date": "2023-07-03",
  "topic": "HR and splats",
  "title": "Re: Splats?",
  "source": "Reddit",
  "author": "boltmarie",
  "sentiment": "negative",
  "summary": "The customer expresses a negative sentiment towards splat points and emphasizes the importance of calorie counting.",
  "text": "I could not give a hoot about splat points and I have been a member for 5 years. I want my calories to be counted.",
  "category": "metric tracking"
} 

13 {
    "date": "2023-07-03",
    "topic": "HR and splats",
    "title": "Re: Splats?",
    "source": "Reddit",
    "author": "graycouch20",
    "sentiment": "positive",
    "summary": "The author has been consistently running in the orange/red heart rate zones for almost 2 years, but they try to ignore splat points and focus on how they feel during the workout.",
    "text": "Even after almost 2 years I still run in the orange/red. I try to ignore splats and focus on how I feel!",
    "category": "workout progress"
} 

14 {
    "date": "20

24 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: How to avoid dropping your HR",
    "source": "Reddit",
    "author": "dabears4hss",
    "sentiment": "positive",
    "summary": "The post discusses the author's strategy of not taking the whole walking recovery and starting back to base once their heart rate reaches the green zone, which they consider a good thing.",
    "text": "When you are healthy, your heart rate will plummet on walking recoveries. So that is a good thing. I usually don't take the whole walking recovery and start back to base once my heart rate gets somewhere in the green and definitely when it hits blue.",
    "category": "workout strategies"
} 

25 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: How to avoid dropping your HR",
    "source": "Reddit",
    "author": "This_Beat2227",
    "sentiment": "positive",
    "summary": "The post provides a helpful suggestion on adjusting walking recovery speed to avoid

34 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: Gray on rowers",
    "source": "Reddit",
    "author": "biker95510",
    "sentiment": "negative",
    "summary": "The author expresses frustration with being in the gray zone while on the rower and prefers to track their workout on their Apple Watch as it is more accurate.",
    "text": "I have the same problem and every time Im on the rower I focus and make sure my grip is loose but Im still in the gray zone. Id rather track my workout on my Apple Watch. Its a lot more accurate",
    "category": "metric tracking"
} 

35 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: Gray on rowers",
    "source": "Reddit",
    "author": "KinvaraSarinth",
    "sentiment": "negative",
    "summary": "The post criticizes the accuracy of armband HRMs on rowers and recommends chest straps for accurate readings.",
    "text": "Armband HRMs are notoriously finicky on rowers. Even with a properly relaxe

48 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "giraffes456",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment and appreciation for the information provided, specifically regarding high max heart rate.",
    "text": "This was really helpful, and comforting. For someone like me with a high max rate.",
    "category": "workout progress"
} 

49 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "little_md",
    "sentiment": "positive",
    "summary": "Glad I could provide some reassurance",
    "text": "Glad I could provide some reassurance",
    "category": "other"
} 

50 {
  "date": "2023-07-02",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "KinvaraSarinth",
  "sentiment": "positive",
  "summary": "The author expresses satisfaction with the accuracy of Ora

61 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "akgray226",
    "sentiment": "positive",
    "summary": "The post expresses satisfaction with Orangetheory's biometric way of measuring workout performance and suggests that the ability to adjust the max heart rate logged in the app contributes to its effectiveness.",
    "text": "They've been able to do it for me twice. I presume its because all the HR data is in the app and thats where it is tracked, they can adjust your max that is logged in the app",
    "category": "metric tracking"
} 

62 {
    "date": "2023-07-01",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "krenzvl",
    "sentiment": "negative",
    "summary": "The customer experienced issues with their heart rate monitor during a class, resulting in incorrect readings and an inability to readjust.",
    "text": "My monitor acted really weird one class

74 {
    "date": "2023-07-01",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "Stock-Shake3915",
    "sentiment": "neutral",
    "summary": "The post is asking for clarification on whether the concern is about heart rate or max heart rate.",
    "text": "Are you asking about not letting your heart rate get that high, or not letting your MHR get that high?",
    "category": "other"
} 

75 {
    "date": "2023-07-01",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "lllrlll11",
    "sentiment": "negative",
    "summary": "The author is venting frustration about Orangetheory increasing the max heart rate, making it harder to achieve splat points.",
    "text": "No. Just venting that they keep making the max go Up.",
    "category": "metric tracking"
} 

76 {
    "date": "2023-07-02",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "Stock-Shake3915",


88 {
    "date": "2023-07-01",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "NextMoose",
    "sentiment": "neutral",
    "summary": "The post/comment discusses how to access information about heart rate zones in the Orangetheory app.",
    "text": "in the app. click on the base heading. scroll, scroll, scroll. click on tell me about my HR Zones",
    "category": "fitness knowledge"
} 

89 {
    "date": "2023-07-01",
    "topic": "HR and splats",
    "title": "Re: Max HR",
    "source": "Reddit",
    "author": "ILoveEm10",
    "sentiment": "positive",
    "summary": "The post/comment expresses positive sentiment and gratitude for finding information.",
    "text": "Found it! Thanks!!",
    "category": "other"
} 

90 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "TennisStraight5521",
  "sentiment": "neutral",
  "summary": "The post/comment expresses a neutral sentiment abo

#### Next 100 posts

In [ ]:
cut_three_month_splat = three_month_splat[401:501]

In [295]:
run_splat_prompt()

0 {
  "date": "2023-07-03",
  "topic": "HR and splats",
  "title": "Re: Splats?",
  "source": "Reddit",
  "author": "This_Beat2227",
  "sentiment": "neutral",
  "summary": "The post/comment provides feedback on the heart rate zones and corrects the percentage for the orange zone.",
  "text": "Orange is 84-91% of max HR (not 75%).",
  "category": "metric tracking"
} 

1 {
  "date": "2023-07-04",
  "topic": "HR and splats",
  "title": "Re: Splats?",
  "source": "Reddit",
  "author": "Cook42523",
  "sentiment": "neutral",
  "summary": "The post is neutral and acknowledges a mistake made by the author.",
  "text": "Hahaha yes youre right! My mistake thank you!",
  "category": "other"
} 

2 {
  "date": "2023-07-03",
  "topic": "HR and splats",
  "title": "Re: Splats?",
  "source": "Reddit",
  "author": "OkRegular167",
  "sentiment": "neutral",
  "summary": "The author expresses a neutral sentiment towards splat points and heart rate zones, stating that they prefer to go off of how they feel

13 {
  "date": "2023-07-03",
  "topic": "HR and splats",
  "title": "Re: Splats?",
  "source": "Reddit",
  "author": "graycouch20",
  "sentiment": "positive",
  "summary": "The author has been consistently running in the orange/red heart rate zones for almost 2 years, but they try to focus on how they feel rather than splat points. They mention getting into the orange zone during warm-up on the treadmill.",
  "text": "Even after almost 2 years I still run in the orange/red. I try to ignore splats and focus on how I feel! Ill typically get in the orange during warm up in base pace on the tread. Lol",
  "category": "workout progress"
} 

14 {
  "date": "2023-07-03",
  "topic": "HR and splats",
  "title": "Re: Splats?",
  "source": "Reddit",
  "author": "Kindly-Might-1879",
  "sentiment": "positive",
  "summary": "The post expresses positive sentiment and highlights the importance of recovery back to green or blue heart rate zones after intense workouts.",
  "text": "Im very in shape and 

25 {
  "date": "2023-07-02",
  "topic": "HR and splats",
  "title": "Re: How to avoid dropping your HR",
  "source": "Reddit",
  "author": "This_Beat2227",
  "sentiment": "positive",
  "summary": "The post provides a positive feedback on how adjusting walking recovery speed can help maintain heart rate in the green zone and promptly return to the orange zone in the next interval.",
  "text": "For me, I wasnt until I figured out how to use my legs properly on the rower that I got to be consistently green with some orange. On tread, I manage my walking recover speed so that my HR doesnt drop below mid green. I go to 3mph initially and then tap up the walking speed in 0.1 increments so that I stay in mid green. This sets me up to get back to orange promptly in the next interval. Green is a large zone from 71-83 % of max HR and so perhaps your walking recovery can be adjusted so that you get to the recovery of green without plummeting ?",
  "category": "workout strategies"
} 

26 {
  "date

36 {
  "date": "2023-07-02",
  "topic": "HR and splats",
  "title": "Re: Gray on rowers",
  "source": "Reddit",
  "author": "sjacobs316",
  "sentiment": "neutral",
  "summary": "The post is asking for advice on whether to switch activities or log them all as one when using an Apple Watch for tracking workouts.",
  "text": "When you use your Apple Watch, do you switch activities or just log it all as one?",
  "category": "other"
} 

37 {
  "date": "2023-07-02",
  "topic": "HR and splats",
  "title": "Re: Gray on rowers",
  "source": "Reddit",
  "author": "Late_Pear1844",
  "sentiment": "neutral",
  "summary": "The post/comment does not provide any specific feedback or sentiment about splat points, max HR, heart rate zones, calorie burn, or other fitness tracking metrics.",
  "text": "I only use an apple watch at OTF and record the whole thing as HIIT",
  "category": "other"
} 

38 {
  "date": "2023-07-02",
  "topic": "HR and splats",
  "title": "Re: Gray on rowers",
  "source": "Reddit"

50 {
  "date": "2023-07-02",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "KinvaraSarinth",
  "sentiment": "positive",
  "summary": "The author expresses satisfaction with the accuracy of the new max HR algorithm and praises its effectiveness in improving their workout experience at Orangetheory. They also criticize the age-based max HR formulas for underestimating their max HR and causing excessive time in the orange and red heart rate zones.",
  "text": "The fact that it was repeatable and consistent over two different HRMs tells me it's accurate. And the new max HR algorithm had been a godsend for me. They currently have it set at 204, which is good enough for OTF classes. All of the age-based max HR formulas are a good 20+ bpm too low, which meant I used to have an awful lot of orange and red time, and regularly topped 100% on AOs.",
  "category": "metric tracking"
} 

51 {
  "date": "2023-07-03",
  "topic": "HR and splats",
  "title": "Re:

64 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "Rosesarepink11",
  "sentiment": "negative",
  "summary": "The author expresses a negative sentiment about their inability to reach a max heart rate of 210, stating that they would be dead if they did.",
  "text": "Mine is 210!! I have never gotten there and I know I won't. Lol I'd be DEAD",
  "category": "workout progress"
} 

65 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "KinvaraSarinth",
  "sentiment": "negative",
  "summary": "The post expresses dissatisfaction with Orangetheory's max HR calculation and age-based formulas, but acknowledges that personalized max HRs work well for some individuals.",
  "text": "The max HR OTF uses is not an exact science. It might be that your max HR is naturally higher. If you feel in control and OK when you hit red, that might be the case. If you don't, then OTF mig

77 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "ulmenreh",
  "sentiment": "positive",
  "summary": "The author mentions having a high green heart rate zone and indicates that they push themselves during the workout, although they don't get many splat points. They also mention that their Fitbit indicates they work hard.",
  "text": "These days coaches comment on the mic if I make it to orange (and I do push myself). I dont get as many splats, but its a good workout, I know I push myself and my Fitbit indicates that I work hard. So high green it is.",
  "category": "workout strategies"
} 

78 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "lllrlll11",
  "sentiment": "positive",
  "summary": "The author expresses satisfaction with their workout performance, consistently achieving 12 splat points and spending a significant amount of time in the green heart 

93 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "heartmydog",
  "sentiment": "negative",
  "summary": "The author stopped wearing their biometric device and prefers to listen to their body instead of focusing on heart rate and splat points.",
  "text": "This is exactly why I stopped wearing mine. I do use my watch but dont focus on it during class. I listen to my body and how I feel.",
  "category": "workout strategies"
} 

94 {
  "date": "2023-07-01",
  "topic": "HR and splats",
  "title": "Re: Max HR",
  "source": "Reddit",
  "author": "TobyRose0207",
  "sentiment": "positive",
  "summary": "The author, a 52-year-old long distance runner, shares their average and highest max heart rate, expressing satisfaction with their performance.",
  "text": "My average is around 126 and my highest max was 152. Im 52 and also a long distance runner and my weight is 227",
  "category": "workout progress"
} 

95 {
  "date": "2023-0

#### Next 100 posts

In [296]:
cut_three_month_splat = three_month_splat[501:601]

In [297]:
run_splat_prompt()

0 {
  "date": "2023-06-16",
  "topic": "HR and splats",
  "title": "Re: Calories VS Splat Points",
  "source": "Reddit",
  "author": "Impressive_Part_6377",
  "sentiment": "neutral",
  "summary": "The author mentions that they burn about 10 calories per minute in the green heart rate zone compared to 12-13 calories in the orange/red zones, and they find it consistent and interesting to track.",
  "text": "I have watched my calories and I burn about 10 per minute in green versus in orange/red its 12-13. I know this is consistent because its something weird I pay attention to to pass the time.",
  "category": "metric tracking"
} 

1 {
  "date": "2023-06-17",
  "topic": "HR and splats",
  "title": "Re: Calories VS Splat Points",
  "source": "Reddit",
  "author": "Stock-Shake3915",
  "sentiment": "positive",
  "summary": "The post expresses positive sentiment towards Orangetheory's biometric way of measuring workout performance, specifically praising the ability to track calories burned by

13 {
  "date": "2023-06-15",
  "topic": "HR and splats",
  "title": "Re: OT Beat band",
  "source": "Reddit",
  "author": "Time-Ad-1594",
  "sentiment": "neutral",
  "summary": "The post is questioning the successful pairing of the OT Beat band to the user's account, indicating a neutral sentiment.",
  "text": "Even though it says I successfully paired it to my account?",
  "category": "other"
} 

14 {
  "date": "2023-06-15",
  "topic": "HR and splats",
  "title": "Re: OT Beat band",
  "source": "Reddit",
  "author": "MBoggles55",
  "sentiment": "negative",
  "summary": "The post/comment has a negative sentiment as the author asks for advice, receives correct advice, but immediately rejects it.",
  "text": "Ask for advice, Receive correct advice, Immediately reject correct advice",
  "category": "other"
} 

15 {
  "date": "2023-06-15",
  "topic": "HR and splats",
  "title": "Re: OT Beat band",
  "source": "Reddit",
  "author": "wiz0rddd",
  "sentiment": "negative",
  "summary": "The au

26 {
  "date": "2023-06-15",
  "topic": "HR and splats",
  "title": "Re: Random heart rate spike during class",
  "source": "Reddit",
  "author": "javedk1",
  "sentiment": "negative",
  "summary": "The author expresses frustration with their heart rate monitor (HRM) not accurately reflecting their heart rate zones during Orangetheory workouts.",
  "text": "I have had this happen and my Apple watch says Im in normal zone Im starting to think my hrm is faulty. Usually changing where I put my hrm has helped alleviate this recently.",
  "category": "metric tracking"
} 

27 {
  "date": "2023-06-15",
  "topic": "HR and splats",
  "title": "Re: Random heart rate spike during class",
  "source": "Reddit",
  "author": "Traditional_Dark_228",
  "sentiment": "negative",
  "summary": "The post expresses concern about a random heart rate spike during class and suggests seeing a cardiologist, while also acknowledging the accuracy of heart rate monitors.",
  "text": "My heart rate does that on occasi

39 {
  "date": "2023-06-11",
  "topic": "HR and splats",
  "title": "Re: Does anyone else's heart rate spike during leg workouts?",
  "source": "Reddit",
  "author": "carmelita93",
  "sentiment": "positive",
  "summary": "The post highlights that larger muscle groups require bigger weights, which in turn demand more from the heart, indicating a positive sentiment towards the effectiveness of Orangetheory's biometric way of measuring workout performance.",
  "text": "Large muscle groups require bigger weights and those weights and muscles demand more from your heart!",
  "category": "workout strategies"
} 

40 {
  "date": "2023-06-11",
  "topic": "HR and splats",
  "title": "Re: Does anyone else's heart rate spike during leg workouts?",
  "source": "Reddit",
  "author": "CuriousSpritual8",
  "sentiment": "positive",
  "summary": "The author expresses excitement about bench step ups increasing their heart rate.",
  "text": "Bench step ups get my heart rate up!",
  "category": "workout st

55 {
  "date": "2023-06-11",
  "topic": "HR and splats",
  "title": "Re: Does anyone else's heart rate spike during leg workouts?",
  "source": "Reddit",
  "author": "livingvikariously",
  "sentiment": "neutral",
  "summary": "The post discusses incline runs and incline PW during leg workouts, but does not provide any specific sentiment or feedback about splat points, max HR, heart rate zones, or other fitness tracking metrics.",
  "text": "And then there were incline runs and incline PW.",
  "category": "other"
} 

56 {
  "date": "2023-06-11",
  "topic": "HR and splats",
  "title": "Re: Does anyone else's heart rate spike during leg workouts?",
  "source": "Reddit",
  "author": "Any-Entrance254",
  "sentiment": "positive",
  "summary": "The post highlights that individuals with more muscle mass may experience higher heart rates during workouts due to increased demand for oxygen and blood flow, comparing them to engines.",
  "text": "We move much more weight up and down compared to lig

70 {
  "date": "2023-06-10",
  "topic": "HR and splats",
  "title": "Re: Updating HRZones",
  "source": "Reddit",
  "author": "almwa123",
  "sentiment": "negative",
  "summary": "The post/comment expresses a negative sentiment towards the limited data provided after only 5 minutes of workout to get a good average.",
  "text": "It's 20 not 5. Not enough data after only 5 to get a good average",
  "category": "metric tracking"
} 

71 {
  "date": "2023-06-10",
  "topic": "HR and splats",
  "title": "Re: Updating HRZones",
  "source": "Reddit",
  "author": "AmanLock",
  "sentiment": "neutral",
  "summary": "The author mentions that their heart rate zones changed after five workouts, indicating a potential update in their fitness level or progress.",
  "text": "Mine changed today. You are right that may not be enough data, but it did change for me today after five.",
  "category": "workout progress"
} 

72 {
  "date": "2023-06-10",
  "topic": "HR and splats",
  "title": "Re: Updating HRZone

85 {
  "date": "2023-06-03",
  "topic": "HR and splats",
  "title": "Re: Zone 3- Green zone",
  "source": "Reddit",
  "author": "Equivalent_Raise8285",
  "sentiment": "neutral",
  "summary": "The author is seeking advice on whether to switch from power walking to jogging in order to achieve more splat points, but expresses concerns about age and joint health.",
  "text": "So it was suggested I begin jogging because I have jogged on an all out be that's like 30-sec to a 1:00 min plus also I think it is because it takes more to get my splat points. Do I want to jog I have mixed feelings, don't get me wrong here age doesn't rule all but I'm 53 I do get stiff and sore sometimes and beating on my joints IDK and yes because I am able to hold a jog. Maybe I do need to push past that shortness of breath feeling. I have been staying in the green zone for a long time as a power walker even with inclines or 6/8 and speeds of 3.9-4.0",
  "category": "workout strategies"
} 

86 {
  "date": "2023-06

96 {
  "date": "2023-06-03",
  "topic": "HR and splats",
  "title": "Re: Zone 3- Green zone",
  "source": "Reddit",
  "author": "umwellshe",
  "sentiment": "neutral",
  "summary": "The author is seeking advice on how to increase their heart rate during workouts and mentions that adding incline helps them reach the orange zone.",
  "text": "I've been having trouble getting out of the base green zone as a runner. I feel like I'm going pretty quick and exerting effort and still I hover around the 81% during flat road. I do notice that when I add the incline to my speed I pop into orange, which is probably why they mentioned to introduce the jogging variable to see if it helps.",
  "category": "workout strategies"
} 

97 {
  "date": "2023-06-04",
  "topic": "HR and splats",
  "title": "Re: Zone 3- Green zone",
  "source": "Reddit",
  "author": "orange7795",
  "sentiment": "positive",
  "summary": "The author expresses annoyance if they get over 20 splats and prefers to stay within the rang

#### Next 100 posts

In [298]:
cut_three_month_splat = three_month_splat[601:]

In [299]:
run_splat_prompt()

0 {
  "date": "2023-06-03",
  "topic": "HR and splats",
  "title": "Re: Zone 3- Green zone",
  "source": "Reddit",
  "author": "Realistic_Warthog_23",
  "sentiment": "positive",
  "summary": "The author mentions that on days when they are worn out mentally or physically, they actually get more splats instead of less, which implies a positive sentiment towards the effectiveness of the biometric measurement system.",
  "text": "On the days when Im worn out mentally or physically, I get *more* splats not less.",
  "category": "workout progress"
} 

1 {
  "date": "2023-06-03",
  "topic": "HR and splats",
  "title": "Re: Zone 3- Green zone",
  "source": "Reddit",
  "author": "fargenable",
  "sentiment": "positive",
  "summary": "The author expresses a positive sentiment about achieving high splat points when consuming sugar.",
  "text": "Its days that I eat sugar that I get the highest splat points.",
  "category": "workout strategies"
} 

2 {
  "date": "2023-06-03",
  "topic": "HR and spla

### Converting the json_objects (Lift45 and HR/splat posts) into a DataFrame

In [300]:
otf_reddit_df = pd.DataFrame(json_objects)

In [301]:
otf_reddit_df

,date,topic,title,source,author,sentiment,summary,text,category
0,2023-07-24,Lift45,Lift 45 TB1 Week of 7/24/23,Reddit,Crafty-Persimmon1857,positive,The author is seeking feedback on the exercises in Lift45 and expresses gratitude for any edits or corrections.,Help a sister out; some of these exercises I feel I put in the wrong block or mislabeled. Let me know what needs an edit. Thanks! Happy lifting. :),exercise content
1,2023-07-24,Lift45,Re: Lift 45 TB1 Week of 7/24/23,Reddit,Fickle-Huckleberry-8,positive,The post expresses positive customer sentiment and compliments about Lift45.,"I was just wishing for this sort of intel, thank you!!!! Looks awesome!!!!",other
2,2023-07-24,Lift45,Re: Lift 45 TB1 Week of 7/24/23,Reddit,backupjesus,positive,The author expresses frustration about missing their regular TB1 class but shows enthusiasm for trying Lift45 at another studio.,I missed my normal TB1 this morning due to a conflicting work call that the other person never showed up for. Grrr. This looks so good I may need to track down a TB1 at another local studio this week. Thanks for the intel OP!,course availability
3,2023-07-23,Lift45,Two different people borrowed from my station today. Lift45 format small rant.,Reddit,cindobeast,negative,"The author expresses frustration with the new format of Lift45 classes, where there are more participants and no designated floor sections, leading to the need to borrow weights and inconvenience.","Before in Lift45 classes we would all have a station and do Floor side first then Rower side exercises. Lately they've allowed more than 16 people to sign up so all floor stations had members plus like 4-5 people who start near the rowers. When we switch those 4-5 people take stations 1-5 and then those folks move to rower side of the gym. So when you don't have a designated floor section you have no choice but to ask people to borrow weights. We had 4 extra people but 2 of them borrowed from mine! They asked and we're nice and all that so it's not a rant against them we'll except one girl who borrowed 20s said she'd bring them back but never did lol. I was able to work with what I could find near me, but if I wanted to go 20s no one near me had them free anyway so it was just a little annoying and I blame it more of this new format than anything. Anyone else have this format at their lift?",course availability
4,2023-07-23,Lift45,Re: Two different people borrowed from my station today. Lift45 format small rant.,Reddit,Caballita14,negative,"The author expresses dissatisfaction with the size of the locations, the lack of weight racks, and the condition of the weight holders in Lift45.",They really need to get bigger locations and have an extra two weight rack. Also fix the weight holders so they arent death claws waiting to cut your fingers.,course availability
...,...,...,...,...,...,...,...,...,...
1176,2023-06-03,HR and splats,Re: Zone 3- Green zone,Reddit,Realistic_Warthog_23,positive,"The author agrees that being uncomfortable at higher heart rates becomes less unpleasant over time, but notes that it may vary depending on the activity. They mention feeling more comfortable at 165 bpm on a treadmill compared to 135 bpm while swimming, attributing it to their experience in running versus being a novice swimmer.","Being uncomfortable is something you get more comfortable with over time. And Im not just talking about getting in better shape. I mean being at higher heart rates becomes (somewhat) less unpleasant the more you do it. But I think its activity specific. Im FAR more comfortable at 165 bpm on a treadmill than I am as a novice swimmer at 135 bpm. The 135 bpm swimming feels like death; the 165 bpm on the tread feels like the nice part before I get going. But thats just bc I have spent a couple decades running and swimming is brand new, I think. But I agree with you: it will get better, slowly, over time, OP!",workout progress
1177,2023-06-03,HR and splats,Re: Zone 3- Green zone,Reddit,Equival

In [303]:
otf_reddit_df.to_csv('/Users/jeanieliu/Desktop/Orangetheory/orangetheory_sentiment/otf_reddit.csv')